# **9장. 추론 최적화**

새로운 모델들이 계속해서 등장하고 사라지지만, 한 가지 변하지 않는 것이 있습니다: 모델을 더 좋게, 더 저렴하게, 더 빠르게 만드는 것입니다. 지금까지 이 책에서는 모델을 더 좋게 만드는 다양한 기법들을 다뤘습니다. 이번 장에서는 모델을 더 빠르고 저렴하게 만드는 데 초점을 맞출 것입니다.

아무리 좋은 모델이라도 너무 느리다면 사용자들이 인내심을 잃을 수 있고, 더 나쁜 경우에는 예측이 무용지물이 될 수 있습니다 - 예를 들어 다음날 주가를 예측하는 모델이 결과를 계산하는 데 이틀이 걸린다고 상상해보세요. 모델이 너무 비싸다면 투자 수익률이 가치가 없을 것입니다.

추론 최적화는 모델, 하드웨어, 서비스 수준에서 수행될 수 있습니다. 모델 수준에서는 학습된 모델의 크기를 줄이거나, 트랜스포머 모델에서 자주 사용되는 어텐션 메커니즘의 계산 병목 현상이 없는 것과 같은 더 효율적인 아키텍처를 개발할 수 있습니다. 하드웨어 수준에서는 더 강력한 하드웨어를 설계할 수 있습니다.

추론 서비스는 사용자 요청을 수용하기 위해 주어진 하드웨어에서 모델을 실행합니다. 특정 하드웨어에 맞게 모델을 최적화하는 기법들을 통합할 수 있습니다. 또한 지연 시간과 비용을 줄이기 위해 리소스를 효율적으로 할당하기 위한 사용량과 트래픽 패턴도 고려해야 합니다.

이러한 이유로 추론 최적화는 모델 연구자, 애플리케이션 개발자, 시스템 엔지니어, 컴파일러 설계자, 하드웨어 아키텍트, 심지어 데이터 센터 운영자들 간의 협업이 자주 일어나는 학제간 분야입니다.

이 장에서는 AI 추론의 병목 현상과 이를 극복하기 위한 기법들을 논의합니다. 주로 모델과 서비스 수준의 최적화에 초점을 맞추며, AI 가속기에 대한 개요도 다룰 것입니다.

이 장에서는 또한 성능 지표와 트레이드오프도 다룹니다. 때로는 모델을 더 빠르게 만드는 기법이 비용도 줄일 수 있습니다. 예를 들어, 모델의 정밀도를 낮추면 크기가 작아지고 더 빨라집니다. 하지만 대개 최적화에는 트레이드오프가 필요합니다. 예를 들어, 최고의 하드웨어는 모델을 더 빠르게 실행할 수 있지만 더 높은 비용이 듭니다.

오픈소스 모델의 가용성이 증가함에 따라 더 많은 팀들이 자체 추론 서비스를 구축하고 있습니다. 하지만 이러한 추론 최적화 기법들을 직접 구현하지 않더라도, 이러한 기법들을 이해하면 추론 서비스와 프레임워크를 평가하는 데 도움이 될 것입니다. 만약 여러분의 애플리케이션의 지연 시간과 비용이 문제가 된다면, 계속 읽어보세요. 이 장이 원인과 잠재적인 해결책을 진단하는 데 도움이 될 것입니다.

 ---

## **추론 최적화 이해하기**

AI 모델의 수명 주기에는 학습과 추론이라는 두 가지 뚜렷한 단계가 있습니다. 학습은 모델을 구축하는 과정을 의미합니다. 추론은 주어진 입력에 대한 출력을 계산하기 위해 모델을 사용하는 과정을 의미합니다. ${ }^{1}$ 모델을 학습하거나 미세 조정하지 않는 한, 대부분 추론에만 신경 쓰면 됩니다. ${ }^{2}$

이 섹션에서는 이 장의 나머지 부분을 논의하기 위한 공통 어휘를 소개하는 추론에 대한 개요로 시작합니다. 이러한 개념들에 이미 익숙하다면, 관심 있는 섹션으로 바로 넘어가셔도 좋습니다.

---


### 추론 개요

프로덕션 환경에서 모델 추론을 실행하는 컴포넌트를 추론 서버라고 합니다. 이는 사용 가능한 모델들을 호스팅하고 필요한 하드웨어에 접근할 수 있습니다. 애플리케이션의 요청(예: 사용자 프롬프트)에 기반하여 적절한 모델을 실행하기 위한 리소스를 할당하고 사용자에게 응답을 반환합니다. 추론 서버는 더 큰 추론 서비스의 일부이며, 이 서비스는 요청을 받고, 라우팅하고, 추론 서버에 도달하기 전에 전처리할 수 있는 책임도 있습니다.

간단한 추론 서비스의 시각화는 그림 9-1에 나와 있습니다.

<img src="./images/fig_09_01.png" width=800>

그림 9-1. 간단한 추론 서비스

OpenAI와 Google이 제공하는 것과 같은 모델 API들은 추론 서비스입니다. 이러한 서비스들을 사용한다면, 이 장에서 논의되는 대부분의 기법들을 구현할 필요가 없습니다. 하지만 모델을 직접 호스팅한다면, 추론 서비스를 구축하고, 최적화하고, 유지보수할 책임이 있습니다.

**계산 병목 현상**

최적화는 병목 현상을 식별하고 해결하는 것입니다. 예를 들어, 교통을 최적화하기 위해 도시 계획자들은 혼잡 지점을 식별하고 혼잡을 완화하기 위한 조치를 취할 수 있습니다. 마찬가지로, 추론 서버는 서비스하는 추론 워크로드의 계산 병목 현상을 해결하도록 설계되어야 합니다. 두 가지 주요 계산 병목 현상이 있는데, 계산 제한과 메모리 대역폭 제한입니다:

- **계산 제한**
    - 이는 작업 완료 시간이 작업에 필요한 계산에 의해 결정되는 작업을 의미합니다. 예를 들어, 암호 해독은 일반적으로 암호화 알고리즘을 해독하는 데 필요한 집중적인 수학적 계산으로 인해 계산 제한적입니다.
- **메모리 대역폭 제한**
    - 이러한 작업들은 시스템 내의 데이터 전송 속도에 의해 제한됩니다(예: 메모리와 프로세서 간의 데이터 이동 속도). 예를 들어, CPU 메모리에 데이터를 저장하고 GPU에서 모델을 학습시킨다면, CPU에서 GPU로 데이터를 이동해야 하는데 이는 오래 걸릴 수 있습니다. 이는 대역폭 제한이라고 줄여 말할 수 있습니다. 문헌에서는 메모리 대역폭 제한을 종종 메모리 제한이라고 합니다.

>**용어 모호성: 메모리 제한 대 대역폭 제한**
>
>메모리 제한은 일부 사람들이 메모리 대역폭이 아닌 메모리 용량에 의해 작업 완료 시간이 제한되는 경우를 지칭하는 데 사용합니다. 이는 하드웨어가 작업을 처리하기에 충분한 메모리가 없을 때 발생합니다. 예를 들어, 기계에 전체 인터넷을 저장할 만큼의 메모리가 없는 경우입니다. 이 메모리는 종종 엔지니어들에게 익숙한 오류로 나타납니다: OOM, 메모리 부족입니다. ${ }^{3}$
>
>하지만 이러한 상황은 종종 작업을 더 작은 조각으로 나누어 완화할 수 있습니다. 예를 들어, GPU 메모리에 제한이 있어 전체 모델을 GPU에 맞출 수 없다면, 모델을 GPU 메모리와 CPU 메모리에 걸쳐 나눌 수 있습니다. 이 분할은 CPU와 GPU 간의 데이터 전송 시간으로 인해 계산 속도를 늦출 것입니다. 하지만 데이터 전송이 충분히 빠르다면, 이는 덜 문제가 됩니다. 따라서 메모리 용량 제한은 실제로는 메모리 대역폭에 관한 것입니다.

계산 제한 또는 메모리 대역폭 제한의 개념은 "Roofline" 논문(Williams et al., 2009)에서 소개되었습니다. ${ }^{4}$ 수학적으로, 연산은 산술 강도(메모리 접근 바이트당 산술 연산 수)에 기반하여 계산 제한 또는 메모리 대역폭 제한으로 분류될 수 있습니다. NVIDIA Nsight와 같은 프로파일링 도구는 그림 9-2와 같이 워크로드가 계산 제한인지 메모리 대역폭 제한인지 알려주는 루프라인 차트를 보여줄 것입니다. 이 차트는 지붕처럼 보이기 때문에 루프라인 차트라고 합니다. 루프라인 차트는 하드웨어 성능 분석에서 흔히 볼 수 있습니다.

서로 다른 최적화 기법들은 서로 다른 병목 현상을 완화하는 것을 목표로 합니다. 예를 들어, 계산 제한 워크로드는 더 많은 칩으로 분산하거나 더 강력한 계산 능력을 가진 칩(예: 더 높은 FLOP/s 수)을 활용하여 속도를 높일 수 있습니다. 메모리 대역폭 제한 워크로드는 더 높은 대역폭을 가진 칩을 활용하여 속도를 높일 수 있습니다.

<img src="./images/fig_09_02.png" width=800>

그림 9-2. 루프라인 차트는 연산이 계산 제한인지 메모리 대역폭 제한인지 시각화하는 데 도움이 됩니다. 이 그래프는 로그 스케일입니다.

서로 다른 모델 아키텍처와 워크로드는 서로 다른 계산 병목 현상을 초래합니다. 예를 들어, Stable Diffusion과 같은 이미지 생성기의 추론은 일반적으로 계산 제한적이며, 자동회귀 언어 모델의 추론은 일반적으로 메모리 대역폭 제한적입니다.

예시로, 언어 모델 추론을 살펴보겠습니다. 2장에서 설명했듯이, 트랜스포머 기반 언어 모델의 추론은 프리필과 디코딩이라는 두 단계로 구성됩니다:

- **프리필**
    - 모델이 입력 토큰들을 병렬로 처리합니다. ${ }^{5}$ 한 번에 처리할 수 있는 토큰의 수는 하드웨어가 주어진 시간에 실행할 수 있는 연산 수에 의해 제한됩니다. 따라서 프리필은 계산 제한적입니다.
- **디코드**
    - 모델이 한 번에 하나의 출력 토큰을 생성합니다. 높은 수준에서 이 단계는 일반적으로 큰 행렬(예: 모델 가중치)을 GPU에 로드하는 것을 포함하며, 이는 하드웨어가 얼마나 빨리 데이터를 메모리에 로드할 수 있는지에 의해 제한됩니다. 따라서 디코딩은 메모리 대역폭 제한적입니다.

그림 9-3은 프리필과 디코딩을 시각화합니다.

<img src="./images/fig_09_03.png" width=800>

그림 9-3. 자동회귀 언어 모델은 추론을 위해 프리필과 디코드라는 두 단계를 따릅니다. <eos>는 시퀀스 끝 토큰을 나타냅니다.

프리필과 디코드는 서로 다른 계산 프로파일을 가지기 때문에, 프로덕션에서는 종종 별도의 기계로 분리됩니다. 이 기법은 "추론 서비스 최적화"에서 논의될 것입니다.

LLM 추론 서버의 프리필과 디코딩 계산량, 따라서 병목 현상에 영향을 미치는 요소들에는 컨텍스트 길이, 출력 길이, 요청 배치 전략이 포함됩니다. 긴 컨텍스트는 일반적으로 메모리 대역폭 제한 워크로드를 초래하지만, 이 장에서 나중에 논의될 영리한 최적화 기법들이 이 병목 현상을 제거할 수 있습니다.

이 글을 쓰는 시점에서, 트랜스포머 아키텍처의 보편성과 기존 가속기 기술의 한계로 인해 많은 AI와 데이터 워크로드가 메모리 대역폭 제한적입니다. 하지만 미래의 소프트웨어와 하드웨어 발전은 AI와 데이터 워크로드를 계산 제한적으로 만들 수 있을 것입니다.

**온라인 및 배치 추론 API**

더 다양한 최적화 기법, 예를 들어 요청을 배치(batch)로 묶거나 더 저렴한 하드웨어를 사용하는 방법도 고려할 수 있습니다. 예를 들어, 이 문서를 작성하는 시점 기준으로 Google Gemini와 OpenAI 모두 배치 API를 제공하며, 이는 비용을 50% 절감해주는 대신 응답 시간이 몇 초 또는 몇 분이 아닌 수 시간 단위로 길어집니다.${ }^{6}$

온라인 API도 지연(latency)에 큰 영향을 주지 않는 한 요청을 배치로 묶어 처리할 수 있습니다. 이 부분은 "Batching" 섹션에서 논의됩니다. 온라인 API와 배치 API의 유일한 차이는 **지연 시간에 대한 초점**입니다. 온라인 API는 낮은 지연 시간에 초점을 맞추는 반면, 배치 API는 높은 처리량(throughput)에 초점을 맞춥니다.

고객을 직접 상대하는(chatbot, 코드 생성 등) 사용 사례는 보통 짧은 지연 시간을 요구하기 때문에 온라인 API를 사용하는 경향이 있습니다. 반대로 지연 시간 요구사항이 덜 엄격한 경우에는 배치 API가 이상적이며, 대표적인 사례는 다음과 같습니다:

- 합성 데이터 생성
- Slack 메시지 요약, 소셜 미디어 브랜드 언급 감성 분석, 고객 지원 티켓 분석 등 주기적인 리포팅
- 신규 고객 온보딩 시 업로드된 모든 문서 처리
- 새로운 모델로 마이그레이션 시 기존 데이터 전체 재처리
- 대규모 고객군을 대상으로 한 개인화 추천 또는 뉴스레터 생성
- 조직의 데이터 재인덱싱을 통한 지식 베이스 업데이트

API는 기본적으로 완성된 응답을 반환합니다. 하지만 **오토리그레시브 디코딩(autoregressive decoding)** 과정에서는 전체 응답이 완성되기까지 시간이 오래 걸릴 수 있으며, 사용자는 기다리는 데 인내심이 없습니다. 그래서 많은 온라인 API는 **스트리밍 모드(streaming mode)** 를 제공하는데, 이는 토큰이 생성될 때마다 순차적으로 반환하는 방식입니다. 이 방법은 첫 번째 토큰이 사용자에게 도달하는 시간을 줄여줍니다. 하지만 이 방식의 단점은 응답 전체를 점수화(score)해 품질을 검토하기 전에 사용자에게 일부 결과가 노출될 수 있다는 점입니다. 물론 이후에 위험이 감지되면, 해당 응답을 수정하거나 제거할 수 있습니다.

> **경고**
>  
> 파운데이션 모델의 배치 API는 전통적인 머신러닝(ML)의 배치 추론(batch inference)과는 다릅니다. 전통적인 ML에서는:
>
> - 온라인 추론(online inference)은 요청이 도착한 후 예측을 계산합니다.
> - 배치 추론(batch inference)은 요청이 도착하기 **전에** 예측을 미리 계산합니다.
>
> 예를 들어, 추천 시스템(recommendation system)처럼 입력이 유한하고 예측 가능한 경우에는 모든 사용자에 대한 추천 결과를 미리 생성해둘 수 있습니다. 그리고 사용자가 웹사이트를 방문할 때 이러한 사전 계산된(predicted) 결과를 제공할 수 있습니다.  
> 하지만 파운데이션 모델의 경우 입력이 개방형(open-ended)이기 때문에, 모든 사용자 입력을 사전에 예측하는 것은 어렵습니다.${ }^{7}$

---

### 추론 성능 지표  

최적화에 들어가기 전에, 최적화할 지표를 이해하는 것이 중요합니다. 사용자 관점에서 중심 축은 지연 시간입니다(응답 품질은 추론 서비스가 아닌 모델 자체의 특성입니다). 그러나 애플리케이션 개발자는 처리량과 활용도도 고려해야 합니다. 이는 애플리케이션의 비용을 결정하기 때문입니다.

**지연 시간, TTFT 및 TPOT**

지연 시간은 사용자가 쿼리를 보낸 시점부터 완전한 응답을 받을 때까지의 시간을 측정합니다. 자기회귀적 생성, 특히 스트리밍 모드에서는 전체 지연 시간이 여러 지표로 나뉠 수 있습니다:

- **첫 번째 토큰까지의 시간(Time to first token)**
    - TTFT는 사용자가 쿼리를 보낸 후 첫 번째 토큰이 생성되는 속도를 측정합니다. 이는 프리필(prefill) 단계의 지속 시간에 해당하며 입력의 길이에 따라 달라집니다. 사용자는 다양한 애플리케이션에 대해 TTFT에 대한 기대치가 다를 수 있습니다. 예를 들어, 대화형 챗봇의 경우 TTFT는 즉각적이어야 합니다.${ }^{8}$ 그러나 사용자는 긴 문서를 요약하는 데 더 오래 기다릴 의향이 있을 수 있습니다.
- **출력 토큰당 시간(Time per output token)**
    - TPOT는 첫 번째 토큰 이후 각 출력 토큰이 생성되는 속도를 측정합니다. 각 토큰이 100ms가 걸린다면, 1,000개 토큰의 응답은 100초가 걸릴 것입니다.
    - 사용자가 생성된 각 토큰을 읽는 스트리밍 모드에서 TPOT는 인간의 읽기 속도보다 빨라야 하지만 그보다 훨씬 빠를 필요는 없습니다. 매우 빠른 독자는 토큰당 $120 \mathrm{~ms}$로 읽을 수 있으므로, 약 120ms의 TPOT, 또는 초당 6-8개 토큰은 대부분의 사용 사례에 충분합니다.
- **토큰 간 시간 및 토큰 간 지연 시간**
    - 이 지표의 변형에는 토큰 간 시간(TBT)과 토큰 간 지연 시간(ITL)이 포함됩니다.${ }^{9}$  둘 다 출력 토큰 사이의 시간을 측정합니다.

총 지연 시간은 `TTFT + TPOT × (출력 토큰 수)`와 같습니다.

같은 총 지연 시간을 가진 두 애플리케이션도 다른 TTFT와 TPOT로 다른 사용자 경험을 제공할 수 있습니다. 사용자는 토큰 사이에 더 긴 대기 시간이 있지만 즉각적인 첫 번째 토큰을 선호할까요, 아니면 첫 번째 토큰을 위해 조금 더 기다리되 그 후 더 빠른 토큰 생성을 즐기는 것을 선호할까요? 최적의 사용자 경험을 결정하기 위해서는 사용자 연구가 필요할 것입니다. 디코딩에서 프리필링으로 더 많은 컴퓨팅 인스턴스를 이동시키거나 그 반대로 함으로써 더 높은 TPOT 비용으로 TTFT를 줄이는 것이 가능합니다.${ }^{10}$ 

사용자가 관찰하는 TTFT 및 TPOT 값은 모델이 관찰하는 값과 다를 수 있다는 점에 유의하는 것이 중요합니다. 특히 모델이 사용자에게 표시되지 않는 중간 단계를 생성하는 CoT(사고의 연쇄) 또는 에이전트 쿼리와 관련된 시나리오에서 그렇습니다. 일부 팀은 사용자가 보는 첫 번째 토큰까지의 시간을 명시적으로 측정하기 위해 '출판까지의 시간(time to publish)'이라는 지표를 사용합니다.

사용자가 쿼리를 보낸 후 모델이 다음 단계를 수행하는 시나리오를 고려해보세요:

1. 일련의 행동으로 구성된 계획을 생성합니다. 이 계획은 사용자에게 표시되지 않습니다.
2. 행동을 취하고 출력을 기록합니다. 이러한 출력은 사용자에게 표시되지 않습니다.
3. 이러한 출력을 기반으로 사용자에게 보여줄 최종 응답을 생성합니다.

모델의 관점에서 첫 번째 토큰은 1단계에서 생성됩니다. 이때 모델은 내부적으로 토큰 생성 프로세스를 시작합니다. 그러나 사용자는 3단계에서 생성된 최종 출력의 첫 번째 토큰만 볼 수 있습니다. 따라서 그들의 관점에서 TTFT는 훨씬 더 깁니다.

지연 시간은 분포이기 때문에 평균은 오해의 소지가 있을 수 있습니다. TTFT 값이 $100 \mathrm{~ms}, 102 \mathrm{~ms}, 100 \mathrm{~ms}, 100 \mathrm{~ms}, 99 \mathrm{~ms}, 104 \mathrm{~ms}, 110 \mathrm{~ms}, 90$ ms, 3,000 ms, 95 ms인 10개의 요청이 있다고 상상해보세요. 평균 TTFT 값은 390ms로, 이는 추론 서비스가 실제보다 느린 것처럼 보이게 합니다. 하나의 요청을 늦춘 네트워크 오류가 있었거나 프리필에 훨씬 더 오랜 시간이 걸린 특히 긴 프롬프트가 있었을 수 있습니다. 어느 쪽이든, 조사해야 합니다. 대량의 요청이 있는 경우, 평균 지연 시간을 왜곡하는 이상치는 거의 불가피합니다.

백분위수로 지연 시간을 보는 것이 더 도움이 됩니다. 이는 요청의 특정 비율에 대해 알려주기 때문입니다. 가장 일반적인 백분위수는 50번째 백분위수(p50, 중앙값)입니다. 중앙값이 100ms라면, 요청의 절반은 첫 번째 토큰을 생성하는 데 100ms보다 더 오래 걸리고, 절반은 100ms보다 적게 걸립니다. 백분위수는 또한 잘못된 것의 증상일 수 있는 이상치를 발견하는 데 도움이 됩니다. 일반적으로, 살펴보고 싶은 백분위수는 p90, p95, p99입니다. 입력 길이에 대한 TTFT 값을 도표로 작성하는 것도 도움이 됩니다.

**처리량과 유효처리량**

처리량은 추론 서비스가 모든 사용자와 요청에 걸쳐 초당 생성할 수 있는 출력 토큰의 수를 측정합니다.

일부 팀은 처리량 계산에 입력 및 출력 토큰을 모두 포함합니다. 그러나 입력 토큰 처리(프리필링)와 출력 토큰 생성(디코딩)은 서로 다른 계산 병목 현상을 가지며 현대 추론 서버에서 종종 분리되어 있기 때문에, 입력 및 출력 처리량을 별도로 계산해야 합니다. 수식어 없이 처리량이라고 할 때는 일반적으로 출력 토큰을 의미합니다.

처리량은 일반적으로 토큰/초(TPS)로 측정됩니다. 여러 사용자를 서비스하는 경우, 사용자당 토큰/초도 시스템이 더 많은 사용자로 확장되는 방식을 평가하는 데 사용됩니다.

처리량은 또한 주어진 시간 동안 완료된 요청 수로 측정될 수 있습니다. 많은 애플리케이션이 초당 요청 수(RPS)를 사용합니다. 그러나 기반 모델 위에 구축된 애플리케이션의 경우, 요청 완료에 몇 초가 걸릴 수 있으므로 많은 사람들이 대신 분당 완료된 요청 수(RPM)를 사용합니다. 이 지표를 추적하는 것은 추론 서비스가 동시 요청을 처리하는 방식을 이해하는 데 유용합니다. 일부 제공업체는 동시에 너무 많은 요청을 보내면 서비스를 제한할 수 있습니다.

처리량은 컴퓨팅 비용과 직접적으로 연결됩니다. 더 높은 처리량은 일반적으로 더 낮은 비용을 의미합니다. 시스템이 컴퓨팅에 시간당 $\$ 2 / \mathrm{h}$의 비용이 들고 처리량이 100 토큰/초인 경우, 출력 토큰 1M당 약 $\$ 5.556$의 비용이 듭니다. 각 요청이 평균 200개의 출력 토큰을 생성한다면, 1K 요청 디코딩 비용은 $\$ 1.11$이 될 것입니다.

프리필 비용도 유사하게 계산할 수 있습니다. 하드웨어 비용이 시간당 $\$ 2$이고 분당 100개의 요청을 프리필할 수 있다면, 1K 요청 프리필링 비용은 $\$ 0.33$이 될 것입니다.

요청당 총 비용은 프리필링과 디코딩 비용의 합입니다. 이 예시에서, 1K 요청의 총 비용은 $\$ 1.11+\$ 0.33=\$ 1.44$입니다.

좋은 처리량으로 간주되는 것은 모델, 하드웨어 및 워크로드에 따라 다릅니다. 작은 모델과 고성능 칩은 일반적으로 더 높은 처리량을 가져옵니다. 일관된 입력 및 출력 길이를 가진 워크로드는 가변 길이를 가진 워크로드보다 최적화하기 쉽습니다.

유사한 크기의 모델, 하드웨어 및 워크로드에 대해서도, 직접적인 처리량 비교는 토큰 수가 토큰을 구성하는 것에 따라 달라지고 다른 모델들은 다른 토크나이저를 가지기 때문에 대략적일 수 있습니다. 요청당 비용과 같은 지표를 사용하여 추론 서버의 효율성을 비교하는 것이 더 좋습니다.

대부분의 다른 소프트웨어 애플리케이션과 마찬가지로, AI 애플리케이션은 지연 시간/처리량 트레이드오프를 가집니다. 배치 처리와 같은 기술은 처리량을 향상시킬 수 있지만 지연 시간을 증가시킵니다. 생성형 AI 제품을 배포한 지 일 년 후 LinkedIn AI 팀의 성찰에 따르면(LinkedIn, 2024), TTFT와 TPOT를 희생할 의향이 있다면 처리량을 두 배 또는 세 배로 늘리는 것은 드문 일이 아닙니다.

이러한 트레이드오프로 인해, 처리량과 비용만을 기반으로 추론 서비스에 집중하면 나쁜 사용자 경험으로 이어질 수 있습니다. 대신, 일부 팀은 LLM 애플리케이션을 위해 네트워킹에서 적응된 지표인 유효처리량(goodput)에 집중합니다. 유효처리량은 SLO(소프트웨어 수준 목표)를 충족하는 초당 요청 수를 측정합니다.

애플리케이션이 다음과 같은 목표를 가지고 있다고 상상해보세요: 최대 200ms의 TTFT와 최대 100ms의 TPOT. 추론 서비스가 분당 100개의 요청을 완료할 수 있다고 가정해 봅시다. 그러나 이 100개의 요청 중 30개만이 SLO를 충족합니다. 그러면 이 서비스의 유효처리량은 분당 30개의 요청입니다. 이에 대한 시각화는 그림 9-4에 나와 있습니다.

<img src="./images/fig_09_04.png" width=800>

그림 9-4. 추론 서비스가 10 RPS를 완료할 수 있지만 그 중 3개만 SLO를 충족한다면, 유효처리량은 3 RPS입니다.

**활용도, MFU 및 MBU**

활용도 지표는 자원이 얼마나 효율적으로 사용되고 있는지 측정합니다. 일반적으로 총 가용 용량에 비해 적극적으로 사용되고 있는 자원의 비율을 정량화합니다.

흔하지만 종종 오해되는 지표는 GPU 활용도이며, NVIDIA는 이러한 오해의 일부 책임이 있습니다. GPU 사용량을 모니터링하기 위한 공식 NVIDIA 도구는 nvidia-smi입니다—SMI는 시스템 관리 인터페이스(System Management Interface)를 의미합니다. 이 도구가 보여주는 하나의 지표는 GPU 활용도로, GPU가 태스크를 적극적으로 처리하는 시간의 비율을 나타냅니다. 예를 들어, GPU 클러스터에서 10시간 동안 추론을 실행하고, GPU가 그 중 5시간 동안 태스크를 적극적으로 처리한다면, GPU 활용도는 50\%가 될 것입니다.

그러나 태스크를 적극적으로 처리한다고 해서 효율적으로 처리한다는 의미는 아닙니다. 단순화를 위해, 초당 100개의 연산을 수행할 수 있는 작은 GPU를 고려해보세요. nvidia-smi의 활용도 정의에 따르면, 이 GPU는 초당 하나의 연산만 수행하더라도 100\% 활용도를 보고할 수 있습니다.

100개의 연산을 수행할 수 있는 기계에 비용을 지불하고 그것을 단 1개의 연산에만 사용한다면, 돈을 낭비하고 있는 것입니다. 따라서 nvidia-smi의 GPU 최적화 지표는 그다지 유용하지 않습니다. 기계가 계산할 수 있는 모든 연산 중에서 주어진 시간에 얼마나 많은 연산을 수행하고 있는지가 여러분이 신경 쓸 수 있는 활용도 지표입니다. 이 지표는 MFU(Model FLOP/s Utilization)라고 불리며, 이는 NVIDIA GPU 활용도 지표와 구별됩니다.

MFU는 관찰된 처리량(tokens/s)과 최대 FLOP/s에서 운영되는 시스템의 이론적 최대 처리량의 비율입니다. 칩 제조업체가 광고하는 최대 FLOP/s에서 칩이 초당 100개의 토큰을 생성할 수 있지만, 추론 서비스에 사용할 때 초당 20개의 토큰만 생성할 수 있다면, MFU는 $20 \% .{ }^{11}$입니다.

마찬가지로, 메모리 대역폭이 비싸기 때문에, 하드웨어의 대역폭이 얼마나 효율적으로 활용되는지도 알고 싶을 수 있습니다. MBU(Model Bandwidth Utilization)는 사용된 달성 가능한 메모리 대역폭의 비율을 측정합니다. 칩의 최대 대역폭이 $1 \mathrm{~TB} / \mathrm{s}$이고 추론에서 $500 \mathrm{~GB} / \mathrm{s}$만 사용한다면, MBU는 $50 \%$입니다.

LLM 추론에 사용되는 메모리 대역폭 계산은 간단합니다:
파라미터 수 $\times$ 바이트/파라미터 $\times$ 토큰/초

MBU는 다음과 같이 계산됩니다:
(파라미터 수 $\times$ 바이트/파라미터 $\times$ 토큰/초) / (이론적 대역폭)

예를 들어, FP16(파라미터당 2바이트)에서 7B 파라미터 모델을 사용하고 초당 100개의 토큰을 달성한다면, 사용된 대역폭은 다음과 같습니다:

$$
7 B \times 2 \times 100=700 \mathrm{~GB} / \mathrm{s}
$$

이는 양자화(7장에서 논의됨)의 중요성을 강조합니다. 파라미터당 더 적은 바이트는 모델이 귀중한 대역폭을 덜 소비한다는 것을 의미합니다.

이것이 이론적으로 $2 \mathrm{~TB} / \mathrm{s}$의 메모리 대역폭을 가진 A100-80GB GPU에서 수행된다면, MBU는 다음과 같습니다:
(700 GB/s) / (2 TB/s) $=70 \%$

처리량(토큰/초)과 MBU 사이의 관계, 그리고 처리량과 MFU 사이의 관계는 선형적이므로, 일부 사람들은 처리량을 사용하여 MBU와 MFU를 참조할 수 있습니다.

좋은 MFU와 MBU로 간주되는 것은 모델, 하드웨어 및 워크로드에 따라 다릅니다. 컴퓨팅 제한 워크로드는 일반적으로 더 높은 MFU와 더 낮은 MBU를 가지며, 대역폭 제한 워크로드는 종종 더 낮은 MFU와 더 높은 MBU를 보여줍니다.

훈련은 더 예측 가능한 워크로드 덕분에 더 효율적인 최적화(예: 더 나은 배치 처리)의 혜택을 받을 수 있기 때문에, 훈련을 위한 MFU는 일반적으로 추론을 위한 MFU보다 높습니다. 추론의 경우, 프리필은 컴퓨팅 제한이고 디코드는 메모리 대역폭 제한이기 때문에, 프리필링 중의 MFU는 일반적으로 디코딩 중의 MFU보다 높습니다. 모델 훈련의 경우, 이 글을 쓰는 시점에서 $50 \%$ 이상의 MFU는 일반적으로 좋은 것으로 간주되지만, 특정 하드웨어에서는 달성하기 어려울 수 있습니다.${ }^{12}$
표 9-1은 여러 모델과 가속기의 MFU를 보여줍니다.

표 9-1. "PaLM: Scaling Language Modeling with Pathways"(Chowdhery 외, 2022)에서의 MFU 예시.

| 모델 | 파라미터 수<br>(십억 단위) | 가속기<br>칩 | 모델<br>FLOP/s<br>활용도 |
| :-- | :-- | :-- | :-- |
| GPT-3 | 175B | V100 | $21.3 \%$ |
| Gopher | 280B | 4096 TPU v3 | $32.5 \%$ |
| Megatron-<br>Turing NLG | 530B | 2240 A100 | $30.2 \%$ |
| PaLM | 540B | 6144 TPU v4 | $46.2 \%$ |

그림 9-5는 다양한 하드웨어에서 FP16으로 Llama 2-70B를 사용한 추론 프로세스의 MBU를 보여줍니다. 감소는 더 많은 사용자로 인한 초당 더 높은 계산 부하로 워크로드가 대역폭 제한에서 컴퓨팅 제한으로 이동하기 때문일 가능성이 높습니다.

<img src="./images/fig_09_05.png" width=800>

그림 9-5. 세 가지 다른 칩에서 FP16 형식의 Llama 2-70B에 대한 대역폭 활용도는 동시 사용자 수가 증가함에 따라 MBU가 감소하는 것을 보여줍니다. 이미지 출처: "LLM Training and Inference with Intel Gaudi 2 AI Accelerators"(Databricks, 2024).

활용도 지표는 시스템의 효율성을 추적하는 데 도움이 됩니다. 동일한 하드웨어에서 유사한 워크로드에 대한 더 높은 활용률은 일반적으로 서비스가 더 효율적이 되고 있음을 의미합니다. 그러나 목표는 가장 높은 활용도를 가진 칩을 얻는 것이 아닙니다. 여러분이 정말로 신경 쓰는 것은 작업을 더 빠르고 저렴하게 완료하는 방법입니다. 비용과 지연 시간이 모두 증가한다면 더 높은 활용률은 아무 의미가 없습니다.

---

### AI 가속기  

소프트웨어가 얼마나 빠르고 저렴하게 실행될 수 있는지는 실행되는 하드웨어에 달려 있습니다. 하드웨어 전반에 걸쳐 작동하는 최적화 기술이 있지만, 하드웨어를 이해하면 더 깊은 최적화가 가능합니다. 이 섹션에서는 추론 관점에서 하드웨어를 살펴보지만, 훈련에도 적용될 수 있습니다.
 
AI 모델과 하드웨어의 발전은 항상 서로 밀접하게 연결되어 왔습니다. 충분히 강력한 컴퓨터의 부족은 1970년대 첫 번째 AI 겨울의 원인 중 하나였습니다. ${ }^{13}$
 
2012년 딥 러닝에 대한 관심의 부활도 컴퓨팅과 밀접하게 연관되어 있었습니다. AlexNet(Krizhevsky 외, 2012)의 인기에 대한 일반적으로 인정되는 이유 중 하나는 신경망을 훈련시키기 위해 GPU(그래픽 처리 장치)를 성공적으로 사용한 최초의 논문이었다는 것입니다. ${ }^{14}$ GPU 이전에는 AlexNet 규모의 모델을 훈련시키려면 Google이 AlexNet 몇 달 전에 출시한 것과 같은 수천 개의 CPU를 사용해야 했습니다. 수천 개의 CPU와 비교하면, 몇 개의 GPU는 PhD 학생들과 연구자들에게 훨씬 더 접근하기 쉬웠고, 이로 인해 딥 러닝 연구 붐이 일어났습니다.

**가속기란 무엇인가?**

가속기는 특정 유형의 계산 워크로드를 가속화하기 위해 설계된 칩입니다. AI 가속기는 AI 워크로드를 위해 설계되었습니다. 가장 지배적인 유형의 AI 가속기는 GPU이며, 2020년대 초 AI 붐 동안의 가장 큰 경제적 원동력은 의심할 여지 없이 NVIDIA입니다.

CPU와 GPU의 주요 차이점은 CPU가 범용 사용을 위해 설계된 반면, GPU는 병렬 처리를 위해 설계되었다는 것입니다:

- CPU는 몇 개의 강력한 코어를 가지고 있으며, 일반적으로 고급 소비자 기계의 경우 최대 64개의 코어를 가집니다. 많은 CPU 코어가 멀티스레드 워크로드를 효과적으로 처리할 수 있지만, 운영 체제 실행, I/O(입력/출력) The text has been cut off.
연산 관리, 또는 복잡하고 순차적인 프로세스 처리와 같이 높은 단일 스레드 성능이 요구되는 작업에서 뛰어납니다.
- GPU는 그래픽 렌더링 및 기계 학습과 같이 많은 작은 독립적인 계산으로 나눌 수 있는 작업에 최적화된 수천 개의 작고 덜 강력한 코어를 가지고 있습니다. 대부분의 ML 워크로드를 구성하는 연산은 행렬 곱셈이며, 이는 고도로 병렬화 가능합니다. ${ }^{15}$

효율적인 병렬 처리의 추구가 계산 능력을 향상시키는 한편, 메모리 설계와 전력 소비에 도전을 제기합니다.

NVIDIA GPU의 성공은 AMD(Advanced Micro Devices)의 새로운 세대 GPU, Google의 TPU(Tensor Processing Unit), Intel의 Habana Gaudi, Graphcore의 Intelligent Processing Unit(IPU), Groq의 Language Processing Unit(LPU), Cerebras의 Wafer-Scale Quant Processing Unit(QPU) 등 AI 워크로드를 가속화하기 위해 설계된 많은 가속기에 영감을 주었으며, 더 많은 가속기가 계속 소개되고 있습니다.

많은 칩이 훈련과 추론을 모두 처리할 수 있지만, 새롭게 떠오르는 큰 주제는 추론을 위한 특수 칩입니다. Desislavov 등(2023)의 조사에 따르면, 추론 비용은 일반적으로 사용되는 시스템에서 훈련 비용을 초과할 수 있으며, 추론은 배포된 AI 시스템의 기계 학습 비용의 최대 $90 \%$를 차지합니다.

7장에서 논의된 바와 같이, 훈련은 역전파 때문에 훨씬 더 많은 메모리를 요구하며 일반적으로 더 낮은 정밀도에서 수행하기가 더 어렵습니다. 또한, 훈련은 보통 처리량을 강조하는 반면, 추론은 지연 시간을 최소화하는 것을 목표로 합니다.

결과적으로, 추론을 위해 설계된 칩은 종종 큰 메모리 용량보다는 더 낮은 정밀도와 더 빠른 메모리 접근을 위해 최적화됩니다. 이러한 칩의 예로는 Apple Neural Engine, AWS Inferentia, MTIA(Meta Training and Inference Accelerator) 등이 있습니다. Google의 Edge TPU와 NVIDIA Jetson Xavier와 같은 엣지 컴퓨팅을 위해 설계된 칩도 일반적으로 추론에 초점을 맞추고 있습니다.

또한 트랜스포머와 같은 특정 모델 아키텍처에 특화된 칩도 있습니다.${ }^{16}$ 많은 칩들이 데이터 센터를 위해 설계되었으며, 점점 더 많은 칩들이 휴대폰이나 노트북과 같은 소비자 기기를 위해 설계되고 있습니다.

다양한 하드웨어 아키텍처는 시간이 지남에 따라 발전하는 다양한 메모리 레이아웃과 특수화된 컴퓨팅 유닛을 가지고 있습니다. 이러한 유닛은 그림 9-6에서 보이는 것처럼 스칼라, 벡터 또는 텐서와 같은 특정 데이터 유형에 최적화되어 있습니다.

<img src="./images/fig_09_06.png" width=800>

그림 9-6. 다양한 계산 프리미티브. Chen 등(2018)에서 영감을 받은 이미지.

칩은 다양한 데이터 유형에 최적화된 다양한 컴퓨팅 유닛의 혼합을 가질 수 있습니다. 예를 들어, GPU는 전통적으로 벡터 연산을 지원했지만, 많은 현대 GPU는 이제 행렬 및 텐서 계산에 최적화된 텐서 코어를 포함합니다. 반면에 TPU는 텐서 연산을 주요 계산 프리미티브로 설계되었습니다. 하드웨어 아키텍처에서 모델을 효율적으로 운영하기 위해서는 그 메모리 레이아웃과 계산 프리미티브를 고려해야 합니다.

칩의 사양에는 각 특정 사용 사례에 대해 이 칩을 평가할 때 유용할 수 있는 많은 세부 정보가 포함되어 있습니다. 그러나 사용 사례 전반에 걸쳐 중요한 주요 특성은 계산 능력, 메모리 크기 및 대역폭, 그리고 전력 소비입니다. 이러한 특성을 설명하기 위해 GPU를 예로 사용하겠습니다.

**계산 능력**

계산 능력은 일반적으로 칩이 주어진 시간에 수행할 수 있는 연산의 수로 측정됩니다. 가장 일반적인 지표는 FLOP/s로, 종종 FLOPS라고 표기되며, 초당 최대 부동 소수점 연산 수를 측정합니다. 그러나 현실적으로는 애플리케이션이 이 최대 FLOP/s를 달성하는 것이 매우 어렵습니다. 실제 FLOP/s와 이론적 FLOP/s 사이의 비율은 하나의 활용도 지표입니다.

칩이 1초 동안 수행할 수 있는 연산의 수는 수치 정밀도에 따라 달라집니다 - 정밀도가 높을수록 칩이 실행할 수 있는 연산은 적어집니다. 두 개의 32비트 숫자를 더하는 것이 일반적으로 두 개의 16비트 숫자를 더하는 것보다 두 배의 계산을 필요로 하는 것을 생각해보세요. 칩이 주어진 시간에 수행할 수 있는 32비트 연산의 수는 서로 다른 칩의 최적화 때문에 16비트 연산의 정확히 절반이 아닙니다. 수치 정밀도에 대한 개요는 "수치 표현"을 다시 살펴보세요.

표 9-2는 NVIDIA H100 SXM 칩의 다양한 정밀도 형식에 대한 FLOP/s 사양을 보여줍니다.

표 9-2. NVIDIA H100 SXM 칩의 FLOP/s 사양.

| 수치<br>정밀도 | 희소성을 가진 teraFLOP/s<br>(1조 FLOP/s) |
| :-- | :-- |
| TF32 텐서 코어$^{\mathrm{a}}$ | 989 |
| BFLOAT16 텐서<br>코어 | 1,979 |
| FP16 텐서 코어 | 1,979 |
| FP8 텐서 코어 | 3,958 |

a. 7장에서 설명했듯이 TF32는 32비트가 아닌 19비트 형식입니다.

**메모리 크기 및 대역폭**

GPU는 병렬로 작동하는 많은 코어를 가지고 있기 때문에, 데이터는 종종 메모리에서 이러한 코어로 이동해야 하며, 따라서 데이터 전송 속도가 중요합니다. 데이터 전송은 큰 가중치 행렬과 학습 데이터를 포함하는 AI 모델로 작업할 때 중요합니다. 이러한 대량의 데이터는 코어를 효율적으로 점유하기 위해 빠르게 이동해야 합니다. 따라서 GPU 메모리는 CPU 메모리보다 더 높은 대역폭과 더 낮은 지연 시간을 가져야 하며, 그렇기 때문에 GPU 메모리는 더 발전된 메모리 기술을 필요로 합니다. 이것은 GPU 메모리가 CPU 메모리보다 더 비싼 요인 중 하나입니다.

더 구체적으로, CPU는 일반적으로 2D 구조를 가진 DDR SDRAM(Double Data Rate Synchronous Dynamic Random-Access Memory)을 사용합니다. GPU, 특히 고급 GPU는 종종 3D 적층 구조를 가진 HBM(고대역폭 메모리)을 사용합니다.${ }^{17}$

가속기의 메모리는 그 크기와 대역폭으로 측정됩니다. 이러한 숫자는 가속기가 속한 시스템 내에서 평가되어야 합니다. GPU와 같은 가속기는 일반적으로 그림 9-7에 시각화된 것처럼 세 가지 수준의 메모리와 상호 작용합니다:

- **CPU 메모리(DRAM)**
    - 가속기는 일반적으로 CPU와 함께 배치되어 CPU 메모리(시스템 메모리, 호스트 메모리 또는 단순히 CPU DRAM으로도 알려짐)에 접근할 수 있습니다.
    - CPU 메모리는 일반적으로 이러한 메모리 유형 중 가장 낮은 대역폭을 가지며, 데이터 전송 속도는 $25 \mathrm{~GB} / \mathrm{s}$에서 $50 \mathrm{~GB} / \mathrm{s}$ 범위입니다. CPU 메모리 크기는 다양합니다. 평균적인 노트북은 약 16-64GB를 가질 수 있으며, 고급 워크스테이션은 1TB 이상을 가질 수 있습니다.
- **GPU 고대역폭 메모리(HBM)**
    - 이것은 GPU에 전용된 메모리로, CPU 메모리보다 더 빠른 접근을 위해 GPU 가까이에 위치합니다.
    - HBM은 상당히 더 높은 대역폭을 제공하며, 데이터 전송 속도는 일반적으로 $256 \mathrm{~GB} / \mathrm{s}$에서 1.5 $\mathrm{TB} / \mathrm{s}$ 이상 범위입니다. 이 속도는 대량의 데이터 전송과 고처리량 작업을 효율적으로 처리하는 데 필수적입니다. 소비자용 GPU는 약 24-80GB의 HBM을 가지고 있습니다.
- **GPU 온칩 SRAM**
    - 칩에 직접 통합된 이 메모리는 거의 즉각적인 접근을 위해 자주 접근하는 데이터와 명령어를 저장하는 데 사용됩니다. 여기에는 SRAM으로 만들어진 L1 및 L2 캐시가 포함되며, 일부 아키텍처에서는 L3 캐시도 포함됩니다. 이러한 캐시는 레지스터 파일과 공유 메모리와 같은 다른 구성 요소도 포함하는 더 넓은 온칩 메모리의 일부입니다.
    - RAM은 종종 $10 \mathrm{~TB} / \mathrm{s}$를 초과하는 매우 높은 데이터 전송 속도를 가집니다. GPU SRAM의 크기는 작으며, 일반적으로 40MB 이하입니다.

<img src="./images/fig_09_07.png" width=800>

그림 9-7. AI 가속기의 메모리 계층 구조. 숫자는 참조용입니다. 실제 숫자는 각 칩마다 다릅니다.

많은 GPU 최적화는 이 메모리 계층 구조를 최대한 활용하는 방법에 관한 것입니다. 그러나 이 글을 쓰는 시점에서, PyTorch와 TensorFlow와 같은 인기 있는 프레임워크는 아직 메모리 접근에 대한 세분화된 제어를 허용하지 않습니다. 이로 인해 많은 AI 연구자와 엔지니어가 CUDA(원래 Compute Unified Device Architecture), OpenAI의 Triton, ROCm(Radeon Open Compute)과 같은 GPU 프로그래밍 언어에 관심을 갖게 되었습니다. 후자는 NVIDIA의 독점 CUDA에 대한 AMD의 오픈 소스 대안입니다.

**전력 소비**

칩은 계산을 수행하기 위해 트랜지스터에 의존합니다. 각 계산은 트랜지스터가 켜지고 꺼지는 것으로 수행되며, 이는 에너지를 필요로 합니다. GPU는 수십억 개의 트랜지스터를 가질 수 있습니다 - NVIDIA A100은 540억 개의 트랜지스터를 가지고 있으며, NVIDIA H100은 800억 개를 가지고 있습니다. 가속기가 효율적으로 사용될 때, 수십억 개의 트랜지스터가 빠르게 상태를 전환하여 상당한 양의 에너지를 소비하고 적지 않은 양의 열을 발생시킵니다. 이 열은 냉각 시스템을 필요로 하며, 이 또한 전기를 소비하여 데이터 센터의 전체 에너지 소비를 증가시킵니다.

칩 에너지 소비는 환경에 엄청난 영향을 미칠 위험이 있으며, 기업이 친환경 데이터 센터를 위한 기술에 투자하도록 하는 압력을 증가시킵니다. 최대 성능으로 1년 동안 실행되는 NVIDIA H100은 약 7,000 kWh를 소비합니다. 비교를 위해, 미국 가정의 연간 평균 전기 소비량은 $10,000 \mathrm{kWh}$입니다. 그래서 전기는 컴퓨팅 확장의 병목 현상입니다.${ }^{18}$

가속기는 일반적으로 최대 전력 소비 또는 대리 지표인 TDP(열 설계 전력)에 따라 전력 소비를 명시합니다:

- 최대 전력 소비는 칩이 최대 부하에서 소비할 수 있는 최대 전력을 나타냅니다.
- TDP는 칩이 일반적인 워크로드에서 작동할 때 냉각 시스템이 방출해야 하는 최대 열을 나타냅니다. 이는 전력 소비의 정확한 측정치는 아니지만, 예상되는 전력 소비의 지표입니다. CPU와 GPU의 경우, 최대 전력 소비는 대략 TDP의 1.1에서 1.5배일 수 있으나, 정확한 관계는 특정 아키텍처와 워크로드에 따라 달라집니다.

클라우드 제공업체를 선택한다면, 냉각이나 전기에 대해 걱정할 필요가 없을 것입니다. 그러나 이러한 숫자들은 여전히 가속기가 환경과 전체 전기 수요에 미치는 영향을 이해하는 데 관심이 될 수 있습니다.

>**가속기 선택**
>
>어떤 가속기를 사용할지는 여러분의 워크로드에 달려 있습니다. 워크로드가 컴퓨팅 제한이라면, 더 많은 FLOP/s를 가진 칩을 찾고 싶을 수 있습니다. 워크로드가 메모리 제한이라면, 더 높은 대역폭과 더 많은 메모리를 가진 칩에 돈을 투자하는 것이 여러분의 삶을 더 쉽게 만들 것입니다.
>
>어떤 칩을 구매할지 평가할 때, 세 가지 주요 질문이 있습니다:
>
>- 하드웨어가 여러분의 워크로드를 실행할 수 있는가?
>- 그렇게 하는 데 얼마나 시간이 걸리는가?
>- 비용은 얼마인가?
>
>FLOP/s, 메모리 크기, 그리고 메모리 대역폭은 처음 두 질문에 답하는 데 도움이 되는 세 가지 큰 숫자입니다. 마지막 질문은 간단합니다. 클라우드 제공업체의 가격은 일반적으로 사용량 기반이며 제공업체 간에 상당히 유사합니다. 하드웨어를 구매한다면, 비용은 초기 가격과 지속적인 전력 소비를 기반으로 계산될 수 있습니다.

---

## **추론 최적화**  

추론 최적화는 모델, 하드웨어 또는 서비스 수준에서 수행될 수 있습니다. 이들의 차이를 설명하기 위해 양궁을 고려해보세요. 모델 수준 최적화는 더 나은 화살을 만드는 것과 같습니다. 하드웨어 수준 최적화는 더 강하고 더 나은 궁수를 훈련시키는 것과 같습니다. 서비스 수준 최적화는 활과 조준 조건을 포함한 전체 사격 과정을 개선하는 것과 같습니다.

이상적으로는, 속도와 비용을 위해 모델을 최적화하는 것이 모델의 품질을 변경해서는 안 됩니다. 그러나 많은 기술들이 모델 성능 저하를 일으킬 수 있습니다. 그림 9-8은 서로 다른 추론 서비스 제공자에 의해 서비스되는 동일한 Llama 모델들의 다양한 벤치마크에서의 성능을 보여줍니다.

<img src="./images/fig_09_08.png" width=800>

그림 9-8. 추론 서비스 제공자는 모델의 동작을 변경할 수 있는 최적화 기술을 사용할 수 있어, 다른 제공자들이 약간의 모델 품질 변동을 갖게 할 수 있습니다. 이 실험은 Cerebras(2024)에 의해 수행되었습니다.

하드웨어 설계는 이 책의 범위를 벗어나므로, 저는 모델과 서비스 수준의 기술에 대해 논의할 것입니다. 기술들이 별도로 논의되지만, 실제 제품에서는 최적화가 일반적으로 하나 이상의 수준에서 기술을 포함한다는 점을 명심하세요.

---

### 모델 최적화

모델 수준 최적화는 모델 자체를 수정하여 더 효율적으로 만드는 것을 목표로 하며, 이는 모델의 동작을 변경할 수 있습니다. 현재 많은 기반 모델들은 트랜스포머 아키텍처를 따르고 자기회귀(autoregressive) 언어 모델 구성 요소를 포함합니다. 이러한 모델들은 추론을 리소스 집약적으로 만드는 세 가지 특성을 가지고 있습니다: 모델 크기, 자기회귀 디코딩, 그리고 어텐션 메커니즘입니다. 이러한 문제들을 해결하기 위한 접근 방식을 살펴보겠습니다.

**모델 압축**

모델 압축은 모델의 크기를 줄이는 기술을 포함합니다. 모델을 더 작게 만들면 더 빠르게 만들 수도 있습니다. 이 책에서는 이미 두 가지 모델 압축 기술을 다루었습니다: 양자화와 증류입니다. 모델의 메모리 사용량을 줄이고 처리량을 늘리기 위해 모델의 정밀도를 낮추는 양자화는 7장에서 다루었습니다. 큰 모델의 동작을 모방하도록 작은 모델을 훈련시키는 모델 증류는 8장에서 다루었습니다.

모델 증류는 더 적은 매개변수로 큰 모델의 동작을 포착할 수 있음을 시사합니다. 큰 모델 내에서 전체 모델의 동작을 포착할 수 있는 매개변수의 하위 집합이 존재할 수 있을까요? 이것이 가지치기(pruning)의 핵심 개념입니다.

신경망 맥락에서 가지치기는 두 가지 의미를 갖습니다. 하나는 신경망의 전체 노드를 제거하는 것으로, 이는 아키텍처를 변경하고 매개변수 수를 줄이는 것입니다. 다른 하나는 예측에 가장 덜 유용한 매개변수를 찾아 0으로 설정하는 것입니다. 이 경우, 가지치기는 총 매개변수 수를 줄이지 않고, 오직 0이 아닌 매개변수의 수만 줄입니다. 이는 모델을 더 희소하게 만들어 모델의 저장 공간을 줄이고 계산 속도를 높입니다.

가지치기된 모델은 그대로 사용하거나 남은 매개변수를 조정하고 가지치기 과정으로 인한 성능 저하를 복원하기 위해 추가적인 미세 조정을 할 수 있습니다. 가지치기는 유망한 모델 아키텍처를 발견하는 데 도움이 될 수 있습니다(Liu et al., 2018). 이러한 가지치기된 아키텍처는 가지치기 전 아키텍처보다 작으며, 처음부터 훈련할 수도 있습니다(Zhu et al., 2017).

문헌에서는 많은 고무적인 가지치기 결과가 있었습니다. 예를 들어, Frankle과 Carbin(2019)은 가지치기 기술이 특정 훈련된 네트워크의 0이 아닌 매개변수 수를 90% 이상 줄일 수 있으며, 정확도를 손상시키지 않으면서 메모리 사용량을 줄이고 속도를 향상시킬 수 있음을 보여주었습니다. 그러나 실제로는 현재 가지치기가 덜 일반적입니다. 원래 모델의 아키텍처에 대한 이해가 필요하기 때문에 더 어렵고, 가져올 수 있는 성능 향상은 종종 다른 접근 방식보다 훨씬 적습니다. 가지치기는 또한 희소 모델을 만들어내며, 모든 하드웨어 아키텍처가 이러한 희소성을 활용하도록 설계되어 있지 않습니다.

가중치 전용 양자화는 사용하기 쉽고, 많은 모델에서 즉시 작동하며, 매우 효과적이기 때문에 가장 인기 있는 접근 방식입니다. 모델의 정밀도를 32비트에서 16비트로 줄이면 메모리 사용량이 절반으로 줄어듭니다. 그러나 우리는 양자화의 한계에 가까워지고 있습니다—값당 1비트보다 더 낮출 수는 없습니다. 증류 역시 많이 사용되는데, 이는 사용자의 요구에 맞게 훨씬 더 큰 모델의 동작과 비교할 수 있는 더 작은 모델을 만들 수 있기 때문입니다.

**자기회귀 디코딩 병목 현상 극복하기**

2장에서 논의한 바와 같이, 자기회귀 언어 모델은 토큰을 하나씩 생성합니다. 하나의 토큰을 생성하는 데 100ms가 걸린다면, 100개의 토큰으로 이루어진 응답은 10초가 걸릴 것입니다.$^{19}$ 이 과정은 느릴 뿐만 아니라 비용도 많이 듭니다. 모델 API 제공자들은 출력 토큰에 입력 토큰보다 약 2~4배의 비용을 부과합니다. Anyscale의 실험에서는 단일 출력 토큰이 100개의 입력 토큰과 동일한 지연 시간을 가질 수 있다는 것을 발견했습니다(Kadous et al., 2023). 자기회귀 생성 과정을 약간만 개선해도 사용자 경험을 크게 향상시킬 수 있습니다.

이 분야는 빠르게 발전하고 있으며, 이 불가능해 보이는 병목 현상을 극복하기 위한 새로운 기술들이 개발되고 있습니다. 어쩌면 언젠가는 이러한 병목 현상이 없는 아키텍처가 나올 수도 있습니다. 여기서 다루는 기술들은 해결책이 어떤 모습일지 보여주기 위한 것이지만, 기술들은 여전히 발전하고 있습니다.

**추측적 디코딩**

추측적 디코딩(speculative decoding, 추측적 샘플링이라고도 함)은 더 빠르지만 덜 강력한 모델을 사용하여 토큰 시퀀스를 생성한 다음, 목표 모델로 이를 검증합니다. 목표 모델은 사용하고자 하는 모델입니다. 더 빠른 모델은 초안(draft) 출력을 제안하기 때문에 초안 모델 또는 제안 모델이라고 불립니다.

입력 토큰이 $x_{1}, x_{2}, \ldots, x_{t}$라고 상상해 보세요:

1. 초안 모델이 $K$개의 토큰 시퀀스를 생성합니다: $x_{t+1}, x_{t+2}$, $\ldots, x_{t+K}$.
2. 목표 모델은 이 $K$개의 생성된 토큰을 병렬로 검증합니다.
3. 목표 모델은 왼쪽에서 오른쪽으로, 사용하기로 동의한 초안 토큰의 가장 긴 하위 시퀀스를 수락합니다.
4. 목표 모델이 $j$개의 초안 토큰 $x_{t+1}, x_{t+2}, \ldots$, $x_{t+j}$를 수락했다고 가정합시다. 그런 다음 목표 모델은 하나의 추가 토큰 $x_{t+j+1}$을 생성합니다.

이 과정은 1단계로 돌아가서, 초안 모델이 $x_{1}, x_{2}, \ldots, x_{t}, x_{t+1}, x_{t+2}, \ldots, x_{t+j}$를 조건으로 $K$개의 토큰을 생성합니다. 이 과정은 그림 9-9에 시각화되어 있습니다.

초안 토큰이 하나도 수락되지 않으면, 이 루프는 목표 모델에 의해 생성된 하나의 토큰만 생성합니다. 모든 초안 토큰이 수락되면, 이 루프는 초안 모델에 의해 생성된 $K$개와 목표 모델에 의해 생성된 1개, 총 $K+1$개의 토큰을 생성합니다.

<img src="./images/fig_09_09.png" width=800>

그림 9-9. 초안 모델이 K개의 토큰 시퀀스를 생성하고, 주 모델은 동의하는 가장 긴 하위 시퀀스를 수락합니다. 이 이미지는 "Blockwise Parallel Decoding for Deep Autoregressive Models"(Stern et al., 2018)에서 가져왔습니다.

모든 초안 시퀀스가 거부되면, 목표 모델은 검증 외에도 전체 응답을 생성해야 하므로 지연 시간이 늘어날 수 있습니다. 그러나 다음 세 가지 통찰력으로 인해 이를 피할 수 있습니다:

1. 목표 모델이 토큰 시퀀스를 검증하는 데 걸리는 시간은 생성하는 데 걸리는 시간보다 적습니다. 검증은 병렬화가 가능한 반면, 생성은 순차적이기 때문입니다. 추측적 디코딩은 디코딩의 계산 프로필을 프리필링(prefilling)의 프로필로 효과적으로 전환합니다.
2. 출력 토큰 시퀀스에서 일부 토큰은 다른 토큰보다 예측하기 쉽습니다. 이러한 예측하기 쉬운 토큰을 올바르게 얻을 수 있는 더 약한 초안 모델을 찾는 것이 가능하여, 초안 토큰의 높은 수락률로 이어질 수 있습니다.
3. 디코딩은 메모리 대역폭에 제한되어 있어, 코딩 프로세스 중에 일반적으로 무료 검증에 사용할 수 있는 유휴 FLOPs가 있습니다.$^{20}$

수락률은 도메인에 따라 다릅니다. 코드와 같이 특정 구조를 따르는 텍스트의 경우 수락률이 일반적으로 더 높습니다. $K$값이 클수록 목표 모델의 검증 호출 횟수가 줄어들지만 초안 토큰의 수락률은 낮아집니다. 초안 모델은 어떤 아키텍처도 가질 수 있지만, 이상적으로는 목표 모델과 동일한 어휘와 토크나이저를 공유해야 합니다. 사용자 정의 초안 모델을 훈련시키거나 기존의 더 약한 모델을 사용할 수 있습니다.

예를 들어, Chinchilla70B의 디코딩 과정을 가속화하기 위해 DeepMind는 동일한 아키텍처의 4B 매개변수 초안 모델을 훈련시켰습니다(Chen et al., 2023). 초안 모델은 목표 모델보다 8배 더 빠르게 토큰을 생성할 수 있습니다(토큰당 $14.1\mathrm{ms}$와 비교하여 토큰당 $1.8\mathrm{ms}$). 이는 응답 품질을 손상시키지 않으면서 전체 응답 지연 시간을 절반 이상 줄입니다. T5-XXL에서도 비슷한 속도 향상이 달성되었습니다(Leviathan et al., 2022).

이 접근 방식은 구현하기 상대적으로 쉽고 모델의 품질을 변경하지 않기 때문에 인기를 얻었습니다. 예를 들어, PyTorch에서 50줄의 코드로 이를 수행할 수 있습니다. 이는 vLLM, TensorRT-LLM, llama.cpp와 같은 인기 있는 추론 프레임워크에 통합되었습니다.

**참조를 통한 추론**

종종 응답은 입력에서 토큰을 참조해야 합니다. 예를 들어, 모델에게 첨부된 문서에 대한 질문을 하면 모델이 문서에서 텍스트 청크를 그대로 반복할 수 있습니다. 또 다른 예는 모델에게 코드의 버그를 수정하도록 요청하면 모델이 원본 코드의 대부분을 약간의 변경사항과 함께 재사용할 수 있습니다. 모델이 이러한 반복되는 토큰을 생성하게 하는 대신, 입력에서 이러한 토큰을 복사하여 생성 속도를 높이면 어떨까요? 이것이 참조를 통한 추론의 핵심 아이디어입니다.

참조를 통한 추론은 추측적 디코딩과 유사하지만, 모델을 사용하여 초안 토큰을 생성하는 대신 입력에서 초안 토큰을 선택합니다. 주요 과제는 각 디코딩 단계에서 컨텍스트에서 가장 관련성 있는 텍스트 구간을 식별하는 알고리즘을 개발하는 것입니다. 가장 간단한 옵션은 현재 토큰과 일치하는 텍스트 구간을 찾는 것입니다.

추측적 디코딩과 달리, 참조를 통한 추론은 추가 모델을 필요로 하지 않습니다. 그러나 검색 시스템, 코딩 또는 다중 턴 대화와 같이 컨텍스트와 출력 사이에 상당한 중복이 있는 생성 시나리오에서만 유용합니다. "Inference with Reference: Lossless Acceleration of Large Language Models"(Yang et al., 2023)에서, 이 기술은 이러한 사용 사례에서 생성 속도를 두 배로 높이는 데 도움이 됩니다.

참조를 통한 추론이 작동하는 방식의 예는 그림 9-10에 나와 있습니다.

<img src="./images/fig_09_10.png" width=800>

그림 9-10. 참조를 통한 추론의 두 가지 예. 입력에서 성공적으로 복사된 텍스트 구간은 빨간색과 녹색으로 표시되어 있습니다. 이미지 출처는 Yang et al. (2023)입니다. 이 이미지는 CC BY 4.0 라이선스 하에 있습니다.

**병렬 디코딩**

초안 토큰으로 자기회귀 생성을 더 빠르게 만드는 대신, 일부 기술은 순차적 의존성을 깨는 것을 목표로 합니다. 기존 토큰 시퀀스 $x_{1}, x_{2}, \ldots, x_{t}$가 주어졌을 때, 이러한 기술은 $x_{t+1}, x_{t+2}, \ldots, x_{t+k}$를 동시에 생성하려고 시도합니다. 이는 모델이 그 앞의 토큰이 $x_{t+1}$이라는 것을 알기 전에 $x_{t+2}$를 생성한다는 것을 의미합니다.

기존 시퀀스에 대한 지식이 종종 다음 몇 개의 토큰을 예측하기에 충분하기 때문에 이것이 작동할 수 있습니다. 예를 들어, "the cat sits"가 주어졌을 때, 다음 토큰이 "on", "under" 또는 "behind"인지 알지 못해도 그 다음 단어가 "the"일 것이라고 예측할 수 있습니다.

병렬 토큰은 Lookahead 디코딩(Fu et al., 2024)에서와 같이 동일한 디코더에 의해 생성되거나, Medusa(Cai et al., 2024)에서와 같이 다른 디코딩 헤드에 의해 생성될 수 있습니다. Medusa에서 원래 모델은 여러 디코딩 헤드로 확장되며, 각 헤드는 특정 위치에서 미래 토큰을 예측하도록 훈련된 작은 신경망 층입니다. 원래 모델이 다음 토큰 $x_{t+1}$을 예측하도록 훈련된 경우, $k^{th}$ 헤드는 토큰 $x_{t+k+1}$을 예측합니다. 이러한 헤드는 원래 모델과 함께 훈련되지만 원래 모델은 고정됩니다. NVIDIA는 Medusa가 HGX H200 GPU에서 Llama 3.1 토큰 생성을 최대 $1.9\times$까지 높이는 데 도움이 되었다고 주장했습니다(Eassa et al., 2024).

그러나 이러한 토큰이 순차적으로 생성되지 않기 때문에, 서로 맞는지 확인하기 위해 검증이 필요합니다. 병렬 디코딩의 필수적인 부분은 검증과 통합입니다. Lookahead 디코딩은 생성된 토큰을 검증하기 위해 Jacobi 방법^{21}을 사용하며, 다음과 같이 작동합니다:

1. K개의 미래 토큰이 병렬로 생성됩니다.
2. 이 $K$개의 토큰은 일관성과 컨텍스트와의 일치성을 검증받습니다.
3. 하나 이상의 토큰이 검증에 실패하면, 모든 $K$개의 미래 토큰을 집계하는 대신 모델은 이러한 실패한 토큰만 재생성하거나 조정합니다.

모델은 모든 토큰이 검증을 통과하고 최종 출력에 통합될 때까지 생성된 토큰을 계속 개선합니다. 이러한 병렬 디코딩 알고리즘 계열은 또한 Jacobi 디코딩이라고도 합니다.

반면, Medusa는 토큰을 검증하고 통합하기 위해 트리 기반 어텐션 메커니즘을 사용합니다. 각 Medusa 헤드는 각 위치에 대해 여러 옵션을 생성합니다. 이러한 옵션은 가장 유망한 조합을 선택하기 위해 트리와 같은 구조로 구성됩니다. 이 과정은 그림 9-11에 시각화되어 있습니다.

<img src="./images/fig_09_11.png" width=800>

그림 9-11. Medusa(Cai et al., 2024)에서 각 헤드는 토큰 위치에 대해 여러 옵션을 예측합니다. 이러한 옵션 중에서 가장 유망한 시퀀스가 선택됩니다. 이미지는 CC BY 4.0 라이선스 하에 있는 논문에서 수정되었습니다.

순차적 의존성을 우회할 수 있다는 관점은 매력적이지만, 병렬 디코딩은 직관적이지 않으며 Medusa와 같은 일부 기술은 구현하기 어려울 수 있습니다.

**어텐션 메커니즘 최적화**

2장에서 살펴본 바와 같이, 다음 토큰을 생성하려면 이전 모든 토큰에 대한 키 벡터와 값 벡터가 필요합니다. 이는 다음을 의미합니다:

- 토큰 $x_{t}$를 생성하려면 토큰 $x_{1}, x_{2}, \ldots, x_{t-1}$에 대한 키 벡터와 값 벡터가 필요합니다.
- 토큰 $x_{t+1}$을 생성하려면 토큰 $x_{1}, x_{2}, \ldots, x_{t-1}, x_{t}$에 대한 키 벡터와 값 벡터가 필요합니다.

토큰 $x_{t+1}$을 생성할 때, 토큰 $x_{1}, x_{2}, \ldots, x_{t-1}$에 대한 키 벡터와 값 벡터를 다시 계산하는 대신, 이전 단계에서 이러한 벡터를 재사용합니다. 이는 가장 최근 토큰 $x_{t}$에 대해서만 키 벡터와 값 벡터를 계산하면 된다는 것을 의미합니다. 재사용을 위해 키 벡터와 값 벡터를 저장하는 캐시를 KV 캐시라고 합니다. 새로 계산된 키 벡터와 값 벡터는 KV 캐시에 추가되며, 이는 그림 9-12에 시각화되어 있습니다.

<img src="./images/fig_09_12.png" width=800>

그림 9-12. 각 디코딩 단계에서 키 벡터와 값 벡터를 재계산하지 않기 위해, 이러한 벡터를 재사용하기 위한 KV 캐시를 사용합니다.

>**참고**
>
>KV 캐시는 훈련 중이 아닌 추론 중에만 사용됩니다. 훈련 중에는 시퀀스의 모든 토큰이 미리 알려져 있기 때문에, 추론 중처럼 순차적으로가 아니라 다음 토큰 생성을 한 번에 계산할 수 있습니다. 따라서 KV 캐시가 필요하지 않습니다.

토큰을 생성하려면 이전의 모든 토큰과 어텐션 점수를 계산해야 하기 때문에, 어텐션 계산의 수는 시퀀스 길이에 따라 지수적으로 증가합니다.^{22} 반면, KV 캐시 크기는 시퀀스 길이에 따라 선형적으로 증가합니다.

KV 캐시 크기는 또한 배치 크기가 커짐에 따라 증가합니다. 구글 논문에 따르면, 멀티 헤드 어텐션, 배치 크기 512, 컨텍스트 길이 2048을 가진 500B+ 모델의 경우, KV 캐시는 총 3TB입니다(Pope et al., 2022). 이는 해당 모델 가중치 크기의 3배입니다.

KV 캐시 크기는 궁극적으로 사용 가능한 하드웨어 저장 공간에 의해 제한되어, 긴 컨텍스트를 가진 애플리케이션을 실행하는 데 병목 현상을 만듭니다. 큰 캐시 크기는 또한 메모리에 로드하는 데 시간이 걸리므로, 엄격한 지연 시간을 가진 애플리케이션에 문제가 될 수 있습니다.

어텐션 메커니즘의 계산 및 메모리 요구 사항은 더 긴 컨텍스트를 갖기 어려운 이유 중 하나입니다.

어텐션 메커니즘을 더 효율적으로 만들기 위해 많은 기술이 개발되었습니다. 일반적으로, 이들은 세 가지 범주로 나뉩니다: 어텐션 메커니즘 재설계, KV 캐시 최적화, 어텐션 계산을 위한 커널 작성.

>**KV 캐시 크기 계산**
>
>최적화 없이 KV 캐시에 필요한 메모리는 다음과 같이 계산됩니다:
>
>- $2 \times B \times S \times L \times H \times M$
>- B: 배치 크기
>- $S$: 시퀀스 길이
>- L: 트랜스포머 레이어 수
>- H: 모델 차원
>- M: 캐시의 숫자 표현에 필요한 메모리(예: FP16 또는 FP32).
>
>이 값은 컨텍스트 길이가 증가함에 따라 상당해질 수 있습니다. 예를 들어, LLama 2 13B는 40개의 레이어와 5,120의 모델 차원을 가집니다. 배치 크기 32, 시퀀스 길이 2,048, 값당 2바이트를 사용하면, 최적화 없이 KV 캐시에 필요한 메모리는 $2 \times 32 \times 2,048 \times 40 \times 5,120 \times 2=54$ GB입니다.

**어텐션 메커니즘 재설계**

이러한 기술은 어텐션 메커니즘의 작동 방식을 변경하는 것과 관련이 있습니다. 이러한 기술이 추론을 최적화하는 데 도움이 되지만, 모델의 아키텍처를 직접 변경하기 때문에 훈련이나 미세 조정 중에만 적용할 수 있습니다.

예를 들어, 새 토큰을 생성할 때, 모든 이전 토큰에 주의를 기울이는 대신, 로컬 윈도우 어텐션은 주변 토큰의 고정 크기 윈도우에만 주의를 기울입니다(Beltagy et al., 2020). 이는 효과적인 시퀀스 길이를 고정 크기 윈도우로 줄여, KV 캐시와 어텐션 계산 모두를 줄입니다. 평균 시퀀스 길이가 10,000 토큰인 경우, 1,000 토큰의 윈도우 크기에 주의를 기울이면 KV 캐시 크기가 10배 줄어듭니다.

로컬 윈도우 어텐션은 글로벌 어텐션과 교차될 수 있으며, 로컬 어텐션은 근처 컨텍스트를 캡처하고, 글로벌 어텐션은 문서 전체에 걸친 작업별 정보를 캡처합니다.

크로스 레이어 어텐션(Brandon et al., 2024)과 멀티 쿼리 어텐션(Shazeer, 2019) 모두 키-값 쌍의 수를 줄임으로써 KV 캐시의 메모리 사용량을 줄입니다. 크로스 레이어 어텐션은 인접한 레이어 간에 키 벡터와 값 벡터를 공유합니다. 동일한 키-값 벡터를 공유하는 세 개의 레이어가 있다는 것은 KV 캐시를 3배 줄인다는 것을 의미합니다. 반면, 멀티 쿼리 어텐션은 쿼리 헤드 간에 키-값 벡터를 공유합니다.

그룹 쿼리 어텐션(Ainslie et al,. 2023)은 멀티 쿼리 어텐션의 일반화입니다. 모든 쿼리 헤드에 대해 하나의 키-값 쌍 세트만 사용하는 대신, 그룹 쿼리 어텐션은 쿼리 헤드를 더 작은 그룹으로 나누고 동일한 그룹 내의 쿼리 헤드 간에만 키-값 쌍을 공유합니다. 이를 통해 쿼리 헤드 수와 키-값 쌍 수 사이에 더 유연한 균형을 맞출 수 있습니다.

AI 챗봇 애플리케이션인 Character.AI는 그들의 평균 대화가 180개의 메시지로 이루어진 대화 기록을 가지고 있다고 공유합니다(2024). 일반적으로 긴 시퀀스를 고려할 때, 추론 처리량의 주요 병목 현상은 KV 캐시 크기입니다. 멀티 쿼리 어텐션, 로컬 어텐션과 글로벌 어텐션의 교차, 크로스 레이어 어텐션이라는 세 가지 어텐션 메커니즘 설계는 KV 캐시를 20배 이상 줄이는 데 도움이 됩니다. 더 중요한 것은, 이러한 상당한 KV 캐시 감소는 큰 배치 크기를 제공하는 데 있어 메모리가 더 이상 병목 현상이 아니라는 것을 의미합니다.

**KV 캐시 크기 최적화**

KV 캐시를 관리하는 방식은 추론 중 메모리 병목 현상을 완화하고 특히 긴 컨텍스트를 가진 애플리케이션에 대해 더 큰 배치 크기를 가능하게 하는 데 중요합니다. KV 캐시를 줄이고 관리하기 위한 많은 기술이 적극적으로 개발되고 있습니다.

가장 빠르게 성장하는 추론 프레임워크 중 하나인 vLLM은 PagedAttention을 도입하여 인기를 얻었습니다. 이는 KV 캐시를 불연속적인 블록으로 나누고, 단편화를 줄이며, 유연한 메모리 공유를 가능하게 하여 LLM 서빙 효율성을 향상시키는 메모리 관리를 최적화합니다(Kwon et al., 2023).

다른 기술로는 KV 캐시 양자화(Hooper et al., 2024; Kang et al., 2024), 적응형 KV 캐시 압축(Ge et al., 2023), 선택적 KV 캐시(Liu et al., 2024) 등이 있습니다.

**어텐션 계산을 위한 커널 작성**

메커니즘 설계를 변경하거나 저장소를 최적화하는 대신, 이 접근 방식은 어텐션 점수가 어떻게 계산되는지 살펴보고 이 계산을 더 효율적으로 만드는 방법을 찾습니다. 이 접근 방식은 계산을 실행하는 하드웨어를 고려할 때 가장 효과적입니다. 특정 칩에 최적화된 코드를 커널이라고 합니다. 커널 작성은 다음 섹션에서 더 자세히 논의될 것입니다.

어텐션 계산에 최적화된 가장 잘 알려진 커널 중 하나는 FlashAttention입니다(Dao et al., 2022). 이 커널은 그림 9-13에 표시된 것처럼 트랜스포머 기반 모델에서 일반적으로 사용되는 많은 연산을 함께 융합하여 더 빠르게 실행되도록 했습니다.

**GPT-2의 어텐션**

<img src="./images/fig_09_13.png" width=800>

그림 9-13. FlashAttention은 여러 공통 연산자를 함께 융합하는 커널입니다. BSD 3-Clause 라이선스 하에 있는 원본 이미지에서 수정되었습니다.

**커널과 컴파일러**

커널은 GPU나 TPU와 같은 특정 하드웨어 가속기에 최적화된 특수 코드 조각입니다. 이들은 일반적으로 이러한 가속기의 성능을 극대화하기 위해 반복적으로, 종종 병렬로 실행해야 하는 계산 집약적인 루틴을 수행하도록 작성됩니다.

행렬 곱셈, 어텐션 계산, 컨볼루션 연산을 포함한 일반적인 AI 연산은 모두 다른 하드웨어에서 계산을 더 효율적으로 만들기 위한 특수 커널을 가지고 있습니다.^{23}

커널을 작성하려면 기본 하드웨어 아키텍처에 대한 깊은 이해가 필요합니다. 여기에는 메모리 계층이 어떻게 구조화되어 있는지(캐시, 글로벌 메모리, 공유 메모리, 레지스터 등)와 데이터가 이러한 다양한 레벨 간에 어떻게 접근되고 이동되는지에 대한 지식이 포함됩니다.

또한, 커널은 일반적으로 CUDA(NVIDIA GPU용), Triton(사용자 정의 커널 작성용으로 OpenAI가 개발한 언어), ROCm(AMD GPU용)과 같은 낮은 수준의 프로그래밍 언어로 작성됩니다. 이러한 언어는 스레드 관리와 메모리 접근에 대한 세밀한 제어를 허용하지만, 대부분의 AI 엔지니어가 친숙한 Python과 같은 언어보다 배우기 어렵습니다.

이러한 진입 장벽으로 인해, 커널 작성은 예전에는 소수에 의해 실행되는 어두운 예술이었습니다. NVIDIA와 AMD와 같은 칩 제조업체는 AI 워크로드에 대한 하드웨어를 효율적으로 만들기 위해 커널을 작성하는 최적화 엔지니어를 고용하는 반면, PyTorch와 TensorFlow와 같은 AI 프레임워크는 다양한 가속기에서 프레임워크를 최적화하기 위해 커널 엔지니어를 고용합니다.

그러나 추론 최적화에 대한 수요 증가와 가속기의 보편화로 인해 더 많은 AI 엔지니어가 커널 작성에 관심을 갖게 되었습니다. 커널 작성에 대한 많은 훌륭한 온라인 튜토리얼이 있습니다. 여기서는 계산 속도를 높이는 데 자주 사용되는 네 가지 일반적인 기술을 다룰 것입니다:

- **벡터화**
    - 루프나 중첩된 루프가 주어졌을 때, 한 번에 하나의 데이터 요소를 처리하는 대신, 메모리에 연속적인 여러 데이터 요소를 동시에 실행합니다. 이는 데이터 I/O 연산을 최소화하여 지연 시간을 줄입니다.
- **병렬화**
    - 입력 배열(또는 n차원 배열)을 서로 다른 코어나 스레드에서 동시에 처리할 수 있는 독립적인 청크로 나누어 계산 속도를 높입니다.
- **루프 타일링**
    - 하드웨어의 메모리 레이아웃과 캐시에 맞게 루프의 데이터 접근 순서를 최적화합니다. 이 최적화는 하드웨어에 의존적입니다. 효율적인 CPU 타일링 패턴은 GPU에서 잘 작동하지 않을 수 있습니다.
- **연산자 융합**
    - 중복 메모리 접근을 피하기 위해 여러 연산자를 단일 패스로 결합합니다. 예를 들어, 두 루프가 동일한 배열에 대해 작동하는 경우, 데이터를 읽고 쓰는 횟수를 줄이기 위해 하나로 융합할 수 있습니다.
    - 벡터화, 병렬화, 루프 타일링은 다양한 모델 전반에 걸쳐 광범위하게 적용될 수 있지만, 연산자 융합은 모델의 특정 연산자와 아키텍처에 대한 더 깊은 이해가 필요합니다. 결과적으로, 연산자 융합은 최적화 엔지니어로부터 더 많은 주의가 필요합니다.

커널은 하드웨어 아키텍처에 최적화되어 있습니다. 이는 새로운 하드웨어 아키텍처가 도입될 때마다 새로운 커널을 개발해야 한다는 것을 의미합니다. 예를 들어, FlashAttention(Dao et al., 2022)은 원래 주로 NVIDIA A100 GPU용으로 개발되었습니다. 이후에는 H100 GPU용 FlashAttention-3가 소개되었습니다(Shah et al., 2024).

모델 스크립트는 해당 모델을 실행하기 위해 수행해야 하는 일련의 연산을 지정합니다. 이 코드를 GPU와 같은 하드웨어 조각에서 실행하려면, 해당 하드웨어와 호환되는 언어로 변환해야 합니다. 이 과정을 '로어링(lowering)'이라고 합니다. 특정 하드웨어에서 실행하기 위해 코드를 로어링하는 도구를 컴파일러라고 합니다. 컴파일러는 ML 모델과 그들이 실행되는 하드웨어 사이를 연결합니다. 로어링 과정 동안, 가능할 때마다, 이러한 연산은 대상 하드웨어에서 더 빠르게 실행하기 위한 특수 커널로 변환됩니다.

> **PYTORCH의 추론 최적화 사례 연구**
>
>그림 9-14는 PyTorch 팀이 다음과 같은 최적화 단계를 통해 Llama-7B에 얼마나 많은 처리량 향상을 줄 수 있는지 보여줍니다(PyTorch. 2023):
>
>1. torch.compile을 호출하여 모델을 더 효율적인 커널로 컴파일합니다.
>2. 모델 가중치를 INT8로 양자화합니다.
>3. 모델 가중치를 추가로 INT4로 양자화합니다.
>4. 추측적 디코딩을 추가합니다.
>
><img src="./images/fig_09_14.png" width=800>
>
>그림 9-14. PyTorch에서 다양한 최적화 기술에 의한 처리량 향상. 이미지 출처는 PyTorch(2023)입니다.
>
>실험은 80GB 메모리를 가진 A100 GPU에서 실행되었습니다. 이러한 최적화 단계가 모델의 출력 품질에 미치는 영향은 불분명했습니다.

컴파일러는 Apache TVM 및 MLIR(Multi-Level Intermediate Representation)와 같은 독립형 도구이거나, torch.compile(PyTorch의 기능), XLA(Accelerated Linear Algebra, 원래 TensorFlow에서 개발되었으며 OpenXLA라는 오픈 소스 버전이 있음), 그리고 NVIDIA GPU에 최적화된 TensorRT에 내장된 컴파일러와 같이 ML 및 추론 프레임워크에 통합될 수 있습니다. AI 기업들은 자체 워크로드 속도를 높이기 위해 설계된 독점 커널이 있는 자체 컴파일러를 가질 수 있습니다.^{24}

---

### **추론 서비스 최적화**

대부분의 서비스 수준 최적화 기술은 리소스 관리에 중점을 둡니다. 고정된 양의 리소스(컴퓨팅 및 메모리)와 동적 워크로드(다양한 모델을 포함할 수 있는 사용자의 추론 요청)가 주어졌을 때, 목표는 지연 시간과 비용을 최적화하기 위해 이러한 워크로드에 리소스를 효율적으로 할당하는 것입니다. 많은 모델 수준 기술과 달리, 서비스 수준 기술은 모델을 수정하지 않으며 출력 품질을 변경해서는 안 됩니다.

**배치 처리**

비용을 줄이는 가장 쉬운 방법 중 하나는 배치 처리입니다. 프로덕션 환경에서 추론 서비스는 동시에 여러 요청을 받을 수 있습니다. 각 요청을 개별적으로 처리하는 대신, 같은 시간대에 도착하는 요청을 함께 배치 처리하면 서비스의 처리량을 크게 줄일 수 있습니다. 각 요청을 개별적으로 처리하는 것이 모든 사람이 자신의 차를 운전하는 것과 같다면, 배치 처리는 그들을 함께 버스에 태우는 것과 같습니다. 버스는 더 많은 사람을 이동시킬 수 있지만, 각 사람의 여정을 더 길게 만들 수도 있습니다. 그러나 지능적으로 수행하면 지연 시간에 미치는 영향을 최소화할 수 있습니다.

배치 처리의 세 가지 주요 기술은 정적 배치 처리, 동적 배치 처리, 그리고 연속 배치 처리입니다.

가장 간단한 배치 처리 기술은 정적 배치 처리입니다. 서비스는 고정된 수의 입력을 배치로 그룹화합니다. 이는 모든 좌석이 채워질 때까지 출발하지 않는 버스와 같습니다. 정적 배치 처리의 단점은 모든 요청이 배치가 가득 찰 때까지 기다려야 실행된다는 것입니다. 따라서 배치의 첫 번째 요청은 마지막 요청이 얼마나 늦게 도착하든 관계없이 배치의 마지막 요청이 도착할 때까지 지연됩니다.

반면, 동적 배치 처리는 각 배치에 대한 최대 시간 창을 설정합니다. 배치 크기가 4이고 창이 100ms인 경우, 서버는 4개의 요청이 있거나 100ms가 경과했을 때 중 먼저 발생하는 조건에 배치를 처리합니다. 이는 고정된 일정에 따라 출발하거나 만석이 되면 출발하는 버스와 같습니다.

이 접근 방식은 지연 시간을 제어하여 이전 요청이 이후 요청에 의해 지연되지 않도록 합니다. 단점은 처리될 때 배치가 항상 가득 차지 않을 수 있어 컴퓨팅 자원이 낭비될 수 있다는 것입니다. 정적 배치 처리와 동적 배치 처리는 그림 9-15에 시각화되어 있습니다.

<img src="./images/fig_09_15.png" width=800>

그림 9-15. 동적 배치 처리는 지연 시간을 관리 가능하게 유지하지만 컴퓨팅 효율성이 떨어질 수 있습니다.

기본적인 배치 처리 구현에서는 모든 배치 요청이 완료되어야 응답이 반환됩니다. LLM의 경우, 일부 요청은 다른 요청보다 훨씬 더 오래 걸릴 수 있습니다. 배치의 한 요청이 10개의 응답 토큰만 생성하고 다른 요청이 1,000개의 응답 토큰을 생성하는 경우, 짧은 응답은 긴 응답이 완료될 때까지 기다렸다가 사용자에게 반환됩니다. 이로 인해 짧은 요청에 불필요한 지연이 발생합니다.

연속 배치 처리는 배치 내의 응답이 완료되는 즉시 사용자에게 반환되도록 합니다. 이는 Orca 논문(Yu et al., 2022)에서 소개된 대로, 한 응답의 생성이 다른 응답을 지연시키지 않는 작업을 선택적으로 배치 처리함으로써 작동합니다. 배치의 요청이 완료되고 응답이 반환된 후, 서비스는 그 자리에 다른 요청을 배치에 추가하여 배치 처리를 연속적으로 만들 수 있습니다. 이는 한 승객을 내려준 후 점유율을 최대화하기 위해 즉시 다른 승객을 태울 수 있는 버스와 같습니다. 인-플라이트 배치 처리라고도 불리는 연속 배치 처리는 그림 9-16에 시각화되어 있습니다.

<img src="./images/fig_09_16.png" width=800>

그림 9-16. 연속 배치 처리를 사용하면 완료된 응답을 즉시 사용자에게 반환할 수 있으며, 해당 위치에서 새 요청을 처리할 수 있습니다.

**프리필과 디코드 분리**

LLM 추론은 프리필(prefill)과 디코드(decode)라는 두 단계로 구성됩니다. 프리필은 컴퓨팅 바운드이고 디코드는 메모리 대역폭 바운드이기 때문에, 동일한 머신을 사용하여 두 가지 모두를 수행하면 리소스를 비효율적으로 경쟁하게 만들어 TTFT(Time to First Token)와 TPOT(Time Per Output Token) 모두를 크게 늦출 수 있습니다. 이미 프리필링과 디코딩을 최대 계산 용량에 가깝게 처리하고 있는 GPU를 상상해 보세요. 이 GPU는 디코딩과 같은 낮은 계산 작업을 처리할 수 있을 것입니다. 그러나 이 GPU에 새 쿼리를 추가한다는 것은 디코딩 작업과 함께 프리필링 작업을 도입한다는 것을 의미합니다. 이 하나의 프리필링 작업은 기존 디코딩 작업에서 계산 리소스를 소모하여 이러한 요청에 대한 TPOT를 늦출 수 있습니다.

추론 서버에 대한 일반적인 최적화 기술 중 하나는 프리필과 디코드를 분리하는 것입니다. "DistServe"(Zhong et al., 2024)와 "Inference Without Interference"(Hu et al., 2024)는 다양한 인기 LLM과 애플리케이션에 대해, 프리필 및 디코드 작업을 다른 인스턴스(예: 다른 GPU)에 할당하면 지연 시간 요구 사항을 준수하면서 처리된 요청 볼륨을 크게 향상시킬 수 있음을 보여줍니다. 분리를 위해 프리필 인스턴스에서 디코드 인스턴스로 중간 상태를 전송해야 하지만, 이 논문은 노드 내의 NVLink와 같은 고대역폭 연결이 있는 현대 GPU 클러스터에서 통신 오버헤드가 상당하지 않음을 보여줍니다.

프리필 인스턴스와 디코드 인스턴스의 비율은 워크로드 특성(예: 더 긴 입력 길이는 더 많은 프리필 계산이 필요함)과 지연 시간 요구 사항(예: TTFT를 낮출지 아니면 TPOT를 낮출지)과 같은 많은 요소에 따라 달라집니다. 예를 들어, 입력 시퀀스가 일반적으로 길고 TTFT를 우선시하려면 이 비율은 2:1에서 4:1 사이일 수 있습니다. 입력 시퀀스가 짧고 TPOT를 우선시하려면 이 비율은 1:2에서 1:1일 수 있습니다.$^{25}$

**프롬프트 캐싱**

많은 애플리케이션의 프롬프트는 중복되는 텍스트 세그먼트를 가지고 있습니다. 프롬프트 캐시는 이러한 중복 세그먼트를 재사용을 위해 저장하므로, 한 번만 처리하면 됩니다. 서로 다른 프롬프트에서 흔히 중복되는 텍스트 세그먼트는 시스템 프롬프트입니다. 프롬프트 캐시가 없으면, 모델은 모든 쿼리마다 시스템 프롬프트를 처리해야 합니다. 프롬프트 캐시를 사용하면 시스템 프롬프트는 첫 번째 쿼리에 대해 한 번만 처리하면 됩니다.

프롬프트 캐싱은 긴 문서를 포함하는 쿼리에 유용합니다. 예를 들어, 많은 사용자 쿼리가 동일한 긴 문서(예: 책이나 코드베이스)와 관련된 경우, 이 긴 문서는 쿼리 간에 재사용하기 위해 캐시될 수 있습니다. 또한 이전 메시지의 처리를 캐시하고 미래 메시지를 예측할 때 재사용할 수 있는 긴 대화에도 유용합니다.

프롬프트 캐시는 그림 9-17에 시각화되어 있습니다. 이는 또한 컨텍스트 캐시 또는 접두사 캐시라고도 불립니다.

<img src="./images/fig_09_17.png" width=800>

그림 9-17. 프롬프트 캐시를 사용하면 서로 다른 프롬프트의 중복 세그먼트를 캐시하고 재사용할 수 있습니다.

긴 시스템 프롬프트가 있는 애플리케이션의 경우, 프롬프트 캐싱은 지연 시간과 비용을 모두 크게 줄일 수 있습니다. 시스템 프롬프트가 1,000 토큰이고, 애플리케이션이 매일 백만 개의 모델 API 호출을 생성한다면, 프롬프트 캐시는 매일 약 10억 개의 반복적인 입력 토큰을 처리하는 것을 절약할 수 있습니다! 그러나 이것은 완전히 무료는 아닙니다. KV 캐시와 마찬가지로, 프롬프트 캐시 크기는 상당히 클 수 있으며 메모리 공간을 차지합니다. 이 기능이 있는 모델 API를 사용하지 않는 한, 프롬프트 캐싱을 구현하려면 상당한 엔지니어링 노력이 필요할 수 있습니다.

2023년 11월 Gim et al.에 의해 소개된 이후, 프롬프트 캐시는 모델 API에 빠르게 통합되었습니다. 이 글을 작성하는 시점에, Google Gemini는 이 기능을 제공하며, 캐시된 입력 토큰은 일반 입력 토큰과 비교하여 75% 할인을 받지만, 캐시 저장에 추가 비용을 지불해야 합니다(작성 시점에, 시간당 100만 토큰당 $1.00). Anthropic은 최대 90%의 비용 절감(캐시된 컨텍스트가 길수록 절감액이 높음)과 최대 75%의 지연 시간 감소를 약속하는 프롬프트 캐싱을 제공합니다. 다양한 시나리오에서 프롬프트 캐싱이 비용과 지연 시간에 미치는 영향은 표 9-3에 나와 있습니다.$^{26}$

표 9-3. 프롬프트 캐싱으로 인한 비용 및 지연 시간 감소. Anthropic(2024)의 정보.

| 사용 사례 | 캐싱 없는 지연 시간(첫 토큰까지 시간) | 캐싱 있는 지연 시간(첫 토큰까지 시간) | 비용 감소 |
| :--: | :--: | :--: | :--: |
| 책과의 채팅 <br> (100,000 토큰 <br> 캐시된 프롬프트) | 11.5초 | 2.4초(-79%) | -90% |
| 다중 샷 프롬프팅(10,000 토큰 프롬프트) | 1.6초 | 1.1초(-31%) | -86% |
| 다중 턴 대화(긴 시스템 프롬프트가 있는 10턴 대화) | ~10초 | ~2.5초(-75%) | -53% |

**병렬화**

가속기는 병렬 처리를 위해 설계되었으며, 병렬화 전략은 고성능 컴퓨팅의 중추입니다. 많은 새로운 병렬화 전략이 개발되고 있습니다. 이 섹션에서는 참조를 위해 그 중 몇 가지만 다룹니다. 모든 모델에 적용할 수 있는 두 가지 병렬화 전략 계열은 데이터 병렬화와 모델 병렬화입니다. LLM에 특별히 적용되는 전략 계열은 컨텍스트 병렬화와 시퀀스 병렬화입니다. 최적화 기술은 여러 병렬화 전략을 포함할 수 있습니다.

복제 병렬화는 구현하기 가장 간단한 전략입니다. 이는 제공하려는 모델의 여러 복제본을 단순히 생성합니다.$^{27}$ 더 많은 복제본을 사용하면 더 많은 칩을 사용하는 비용으로 더 많은 요청을 동시에 처리할 수 있습니다. 다양한 크기의 모델을 다른 칩에 맞추려는 시도는 빈 패킹 문제이며, 더 많은 모델, 더 많은 복제본, 더 많은 칩이 있을수록 복잡해질 수 있습니다.

다양한 크기의 모델(예: 8B, 13B, 34B, 70B 매개변수)과 다양한 메모리 용량을 가진 GPU(예: 24 GB, 40 GB, 48 GB, 80 GB)에 접근할 수 있다고 가정해 보겠습니다. 단순화를 위해 모든 모델이 동일한 정밀도인 8비트라고 가정합니다:

- 고정된 수의 칩이 있는 경우, 각 모델에 대해 몇 개의 복제본을 생성하고 각 복제본에 어떤 GPU를 사용할지 결정하여 메트릭을 최대화해야 합니다. 예를 들어, 40 GB GPU에 세 개의 13B 모델을 배치해야 할까요, 아니면 이 GPU를 하나의 34B 모델용으로 예약해야 할까요?
- 고정된 수의 모델 복제본이 있는 경우, 비용을 최소화하기 위해 어떤 칩을 획득해야 하는지 결정해야 합니다. 그러나 이러한 상황은 거의 발생하지 않습니다.

종종, 모델이 너무 커서 하나의 기계에 맞지 않을 수 있습니다. 모델 병렬화는 동일한 모델을 여러 기계에 분할하는 관행을 의미합니다. 모델 병렬화를 사용하면 칩에 모델을 맞추는 것이 더 복잡한 문제가 될 수 있습니다.

모델을 분할하는 여러 가지 방법이 있습니다. 추론에 가장 일반적인 접근 방식은 텐서 병렬화(연산자 내부 병렬화라고도 함)입니다. 추론은 행렬 곱셈과 같은 다차원 텐서에 대한 연산자 시퀀스를 포함합니다. 이 접근 방식에서는 연산자에 관련된 텐서가 여러 장치에 분할되어 이 연산자를 병렬로 실행할 수 있는 더 작은 조각으로 효과적으로 분해하여 계산 속도를 높입니다. 예를 들어, 두 행렬을 곱할 때, 그림 9-18에 표시된 것처럼 행렬 중 하나를 열 방향으로 분할할 수 있습니다.

텐서 병렬화는 두 가지 이점을 제공합니다. 첫째, 단일 기계에 맞지 않는 대형 모델을 제공할 수 있게 합니다. 둘째, 지연 시간을 줄입니다. 그러나 지연 시간 이점은 추가 통신 오버헤드로 인해 감소할 수 있습니다.

<img src="./images/fig_09_18.png" width=800>

그림 9-18. 행렬 곱셈을 위한 텐서 병렬화.

모델을 분할하는 또 다른 방법은 파이프라인 병렬화로, 모델의 계산을 구별된 단계로 나누고 각 단계를 다른 장치에 할당하는 것을 포함합니다. 데이터가 모델을 통해 흐를 때, 각 단계는 한 부분을 처리하는 동안 다른 단계는 후속 부분을 처리하여 계산을 중첩시킬 수 있습니다. 그림 9-19는 네 대의 기계에서 파이프라인 병렬화가 어떻게 보이는지 보여줍니다.

<img src="./images/fig_09_19.png" width=800>

그림 9-19. 파이프라인 병렬화는 모델 분할이 병렬로 실행되도록 합니다.

그림 9-19는 배치가 더 작은 마이크로배치로 분할될 수 있음을 보여줍니다. 한 기계에서 마이크로배치가 처리된 후, 그 출력은 다음 기계의 모델의 다음 부분으로 전달됩니다.

파이프라인 병렬화는 여러 기계에서 대형 모델을 제공할 수 있게 하지만, 파이프라인 단계 간의 추가 통신으로 인해 각 요청에 대한 총 지연 시간을 증가시킵니다. 따라서 엄격한 지연 시간 요구 사항이 있는 애플리케이션의 경우, 파이프라인 병렬화는 일반적으로 복제 병렬화를 선호하여 피합니다. 그러나 파이프라인 병렬화는 처리량을 증가시키는 데 도움이 될 수 있으므로 훈련에서 일반적으로 사용됩니다.

덜 일반적이지만 기술의 다양성을 보여주기 위해 언급할 만한 두 가지 기술은 컨텍스트 병렬화와 시퀀스 병렬화입니다. 이들은 모두 긴 입력 시퀀스 처리를 더 효율적으로 만들기 위해 개발되었습니다.

컨텍스트 병렬화에서는 입력 시퀀스 자체가 다른 장치에 분할되어 별도로 처리됩니다. 예를 들어, 입력의 첫 번째 절반은 기계 1에서 처리되고 두 번째 절반은 기계 2에서 처리됩니다.

시퀀스 병렬화에서는 전체 입력에 필요한 연산자가 기계 간에 분할됩니다. 예를 들어, 입력에 어텐션과 피드포워드 계산이 모두 필요한 경우, 어텐션은 기계 1에서 처리되고 피드포워드는 기계 2에서 처리될 수 있습니다.

---

# **요약**

모델의 사용성은 추론 비용과 지연 시간에 크게 의존합니다. 더 저렴한 추론은 AI 기반 결정을 더 경제적으로 만들고, 더 빠른 추론은 더 많은 애플리케이션에 AI를 통합할 수 있게 합니다. 추론 최적화의 막대한 잠재적 영향을 고려할 때, 이는 혁신적인 접근 방식을 지속적으로 고안하는 많은 재능 있는 개인들을 끌어들였습니다.

효율성을 높이기 전에, 효율성이 어떻게 측정되는지 이해해야 합니다. 이 장은 지연 시간, 처리량 및 활용도에 대한 일반적인 효율성 메트릭으로 시작했습니다. 언어 모델 기반 추론의 경우, 지연 시간은 프리필링 단계의 영향을 받는 첫 토큰까지의 시간(TTFT)과 디코딩 단계의 영향을 받는 출력 토큰당 시간(TPOT)으로 나눌 수 있습니다. 처리량 메트릭은 비용과 직접적으로 관련이 있습니다. 지연 시간과 처리량 사이에는 트레이드오프가 있습니다. 지연 시간 증가를 허용할 수 있다면 비용을 잠재적으로 줄일 수 있으며, 지연 시간을 줄이는 것은 종종 비용 증가를 수반합니다.

모델이 얼마나 효율적으로 실행될 수 있는지는 실행되는 하드웨어에 달려 있습니다. 이러한 이유로, 이 장에서는 AI 하드웨어와 다양한 가속기에서 모델을 최적화하는 데 필요한 것에 대한 간략한 개요도 제공했습니다.

이어서 이 장은 추론 최적화를 위한 다양한 기술을 계속 다루었습니다. 모델 API의 가용성을 고려할 때, 대부분의 애플리케이션 개발자는 이러한 기술을 직접 구현하는 대신 내장된 최적화가 있는 이러한 API를 사용할 것입니다. 이러한 기술이 모든 애플리케이션 개발자에게 관련이 있지 않을 수 있지만, 어떤 기술이 가능한지 이해하는 것은 모델 API의 효율성을 평가하는 데 도움이 될 수 있다고 생각합니다.

이 장은 또한 모델 수준과 추론 서비스 수준에서의 최적화에 중점을 두었습니다. 모델 수준 최적화는 종종 모델 자체를 변경해야 하며, 이는 모델 동작의 변화로 이어질 수 있습니다. 반면, 추론 서비스 수준 최적화는 일반적으로 모델을 그대로 유지하고 서비스되는 방식만 변경합니다.

모델 수준 기술에는 양자화 및 증류와 같은 모델에 구애받지 않는 기술이 포함됩니다. 다양한 모델 아키텍처는 각자의 최적화를 필요로 합니다. 예를 들어, 트랜스포머 모델의 주요 병목 현상은 어텐션 메커니즘에 있기 때문에, KV 캐시 관리 및 어텐션 커널 작성을 포함하여 어텐션을 더 효율적으로 만드는 많은 최적화 기술이 있습니다. 자기회귀 언어 모델의 큰 병목 현상은 자기회귀 디코딩 프로세스에 있으며, 결과적으로 이를 해결하기 위한 많은 기술도 개발되었습니다.

추론 서비스 수준 기술에는 다양한 배치 처리 및 병렬화 전략이 포함됩니다. 또한 프리필링/디코딩 분리 및 프롬프트 캐싱을 포함하여 자기회귀 언어 모델을 위해 특별히 개발된 기술도 있습니다.

최적화 기술의 선택은 워크로드에 따라 달라집니다. 예를 들어, KV 캐싱은 짧은 컨텍스트보다 긴 컨텍스트가 있는 워크로드에 훨씬 더 중요합니다. 반면, 프롬프트 캐싱은 긴 중복 프롬프트 세그먼트나 다중 턴 대화를 포함하는 워크로드에 중요합니다. 선택은 또한 성능 요구 사항에 따라 달라집니다. 예를 들어, 낮은 지연 시간이 비용보다 우선순위가 높다면, 복제 병렬성을 확장하고 싶을 수 있습니다. 더 많은 복제본은 추가 기계를 필요로 하지만, 각 기계는 더 적은 요청을 처리하여 요청당 더 많은 리소스를 할당하고, 따라서 응답 시간을 개선할 수 있습니다.

그러나 다양한 사용 사례에서, 가장 영향력 있는 기술은 일반적으로 양자화(일반적으로 모델 전반에 걸쳐 잘 작동함), 텐서 병렬화(지연 시간을 줄이고 더 큰 모델 제공을 가능하게 함), 복제 병렬화(구현이 비교적 간단함), 그리고 어텐션 메커니즘 최적화(트랜스포머 모델을 크게 가속화할 수 있음)입니다.

추론 최적화는 이 책에서 다루는 모델 적응 기술 목록을 마무리합니다. 다음 장에서는 이러한 기술들을 어떻게 일관된 시스템으로 통합하는지 살펴볼 것입니다.재시도Claude는 실수를 할 수 있습니다. 응답을 반드시 다시 확인해 주세요.

---

1. 7장에서 논의한 바와 같이, 추론은 순방향 전달을 포함하는 반면 훈련은 순방향 및 역방향 전달을 모두 포함합니다.
2. 친구인 Mark Saroufim이 모델의 훈련 비용과 추론 비용 사이의 흥미로운 관계를 알려주었습니다. 당신이 모델 제공자라고 상상해 보세요. $T$를 총 훈련 비용, $p$를 추론당 청구하는 비용, $N$을 판매할 수 있는 추론 호출 수라고 합시다. 모델 개발은 모델에 대한 추론으로부터 회수할 수 있는 돈이 훈련 비용보다 많을 때만 의미가 있습니다, 즉 $T<=p \times N$입니다. 모델이 프로덕션에서 더 많이 사용될수록, 모델 제공자는 추론 비용을 더 줄일 수 있습니다. 그러나 이는 오픈 소스 모델 위에 추론 호출을 판매하는 제3자 API 제공자에게는 적용되지 않습니다.
3. 일화적으로, 저는 시스템 배경을 가진 사람들(예: 최적화 엔지니어 및 GPU 엔지니어)이 메모리 바운드를 대역폭 바운드를 지칭하는 데 사용하고, AI 배경을 가진 사람들(예: ML 및 AI 엔지니어)이 메모리 바운드를 메모리 용량 바운드를 지칭하는 데 사용한다는 것을 발견했습니다.
4. Roofline 논문은 메모리 바운드라는 용어를 메모리 대역폭 바운드를 지칭하는 데 사용합니다.
5. 프리필링은 트랜스포머 모델을 위한 초기 KV 캐시를 효과적으로 채웁니다.
6. 추론 서비스를 운영한다면, 추론 API를 온라인과 배치로 분리하면 지연 시간이 가장 중요한 요청에 대해 지연 시간을 우선시할 수 있습니다. 당신의 추론 서버가 지연 시간 저하 없이 최대 X 요청/초만 제공할 수 있고, Y 요청/초를 제공해야 하며, Y가 X보다 크다고 가정해 봅시다. 이상적인 세계에서는 덜 긴급한 요청을 가진 사용자가 배치 API에 요청을 보낼 수 있어, 서비스가 온라인 API 요청을 먼저 처리하는 데 집중할 수 있습니다.
7. "프롬프트 캐싱"에서 논의된 바와 같이, 애플리케이션의 시스템 프롬프트를 미리 아는 것이 일반적입니다. 예측하기 어려운 것은 정확한 사용자 쿼리입니다.
8. 챗봇 초기에는 일부 사람들이 챗봇이 너무 빨리 응답하는 것이 부자연스럽다고 불평했습니다. "Lufthansa Delays Chatbot's Responses to Make It More 'Human'" (Ry Crozier, iTnews, 2017년 5월)을 참조하세요. 그러나 사람들이 챗봇에 더 익숙해짐에 따라 이는 더 이상 그렇지 않습니다.
9. 토큰 간 시간(TBT)은 LinkedIn에서 사용되고 토큰 간 지연 시간(ITL)은 NVIDIA에서 사용됩니다.
10. Anyscale의 실험에 따르면 100개의 입력 토큰이 전체 지연 시간에 미치는 영향은 단일 출력 토큰과 대략적으로 동일합니다.
11. 사람들은 오랫동안 FLOP/s 활용도에 관심을 가져왔지만, MFU라는 용어는 PaLM 논문(Chowdhery et al., 2022)에서 소개되었습니다.
12. 칩 제조업체는 제가 피크 FLOP/s 해킹이라고 부르는 것을 할 수도 있습니다. 이는 특정 형태의 희소 행렬을 사용하는 등 특정 조건에서 실험을 실행하여 피크 FLOP/s를 증가시킬 수 있습니다. 더 높은 피크 FLOP/s 수치는 칩을 더 매력적으로 만들지만, 사용자가 높은 MFU를 달성하기는 더 어려울 수 있습니다.
13. 1960년대에 컴퓨터는 매우 제한된 기능을 가진 단일 계층 신경망만 실행할 수 있었습니다. 그들의 유명한, 1969년 책 "Perceptrons: An Introduction to Computational Geometry" (MIT Press)에서, 두 AI 선구자인 Marvin Minsky와 Seymour Papert는 은닉층이 있는 신경망도 여전히 거의 할 수 있는 것이 없을 것이라고 주장했습니다. 그들의 정확한 인용문은 다음과 같습니다: "이후 종류의 기계의 계산 능력에 대해서는 사실상 알려진 것이 없습니다. 우리는 그것이 저차 퍼셉트론보다 조금 더 할 수 있다고 믿습니다." 그들의 주장을 반박할 충분한 계산력이 없었고, 이는 그 후 1970년대에 AI 자금이 고갈된 주요 이유로 많은 사람들에 의해 인용되었습니다.
14. GPU는 그래픽 이상으로 많이 사용되기 때문에 이름을 바꾸는 것에 대한 논의가 있었습니다(Jon Peddie, "Chasing Pixels," 2018년 7월). NVIDIA의 CEO인 Jensen Huang은 인터뷰(Stratechery, 2022년 3월)에서 GPU가 성공하고 여기에 더 많은 기능을 추가한 후, GPGPU(일반 목적 GPU) 또는 XGU와 같은 더 일반적인 이름으로 변경하는 것을 고려했다고 말했습니다. 그들은 GPU를 구매하는 사람들은 이름 이상으로 GPU가 무엇에 좋은지 알 만큼 똑똑할 것이라고 가정했기 때문에 이름 변경에 반대했습니다.
15. 행렬 곱셈(애정을 담아 matmul이라고 불림)은 "Data Movement Is All You Need: A Case Study on Optimizing Transformers" (Ivanov et al., arXiv, v3, 2021년 11월)과 "Scalable MatMul-free Language Modeling" (Zhu et al., arXiv, 2024년 6월)에 따르면 신경망의 모든 부동 소수점 연산의 90% 이상을 차지하는 것으로 추정됩니다.
16. 칩은 한 가지 모델 아키텍처를 실행하도록 개발될 수 있지만, 모델 아키텍처도 칩을 최대한 활용하도록 개발될 수 있습니다. 예를 들어, 트랜스포머는 원래 Google이 TPU에서 빠르게 실행되도록 설계했으며 나중에 GPU에서 최적화되었습니다.
17. 저가에서 중간급 GPU는 GDDR(Graphics Double Data Rate) 메모리를 사용할 수 있습니다.
18. 수만 개의 GPU로 데이터 센터를 구축하는 주요 과제는 필요한 전기를 보장할 수 있는 위치를 찾는 것입니다. 대규모 데이터 센터를 구축하려면 전기 공급, 속도 및 지정학적 제약 조건을 탐색해야 합니다. 예를 들어, 원격 지역은 더 저렴한 전기를 제공할 수 있지만 네트워크 지연 시간을 증가시킬 수 있어 추론과 같은 엄격한 지연 시간 요구 사항이 있는 사용 사례에 대한 데이터 센터의 매력을 떨어뜨릴 수 있습니다.
19. 각 토큰 생성 단계는 전체 모델의 매개변수를 가속기의 고대역폭 메모리에서 계산 유닛으로 전송해야 합니다. 이는 이 작업을 대역폭이 많이 소모되게 만듭니다. 모델은 한 번에 하나의 토큰만 생성할 수 있기 때문에, 이 과정은 소수의 FLOP/s만 소비하여 계산 비효율성을 초래합니다.
20. 이는 또한 MFU가 이미 최대치에 도달했다면 추측적 디코딩이 덜 의미가 있다는 것을 의미합니다.
21. Jacobi 방법은 솔루션의 여러 부분이 동시에 독립적으로 업데이트될 수 있는 반복적 알고리즘입니다.
22. 자기회귀 모델의 어텐션 계산 수는 $O\left(n^{2}\right)$입니다.
23. 컨볼루션 연산은 종종 Stable Diffusion과 같은 이미지 생성 모델에서 사용됩니다.
24. 많은 기업들은 자신들의 커널을 영업 비밀로 간주합니다. 경쟁자보다 모델을 더 빠르고 저렴하게 실행할 수 있는 커널을 보유하는 것은 경쟁 우위입니다.
25. 프리필과 디코드 인스턴스 비율을 언급하는 강연에는 "Llama Inference at Meta" (Meta, 2024)가 포함됩니다.
26. llama.cpp도 프롬프트 캐싱을 가지고 있지만, 이 글을 쓰는 시점에서는 전체 프롬프트만 캐시하고 동일한 채팅 세션의 쿼리에 대해서만 작동하는 것 같습니다. 문서는 제한적이지만, 코드를 읽어본 제 추측으로는 긴 대화에서 이전 메시지를 캐시하고 가장 최근 메시지만 처리합니다.
27. 훈련 중에는 동일한 기술을 데이터 병렬화라고 합니다.